In [1]:
# Import dependencies.
import pandas as pd
import numpy as np
import os
import dask.dataframe as dd
from tqdm.auto import tqdm
pd.set_option('display.max_columns', 500)

In [2]:
# Set path to files and read in the CSV file. Use dtyping to save memory and use dask for efficiency.
path = 'C:/Users/outla/Desktop/VA_ML_Research/UCSD_NRD_2017'
os.chdir(path)
core_file = dd.read_csv("Nghia_NRD_2017.csv", dtype={'age':'float16', 'aweekend':'float16', 'died':'float16', 'discwt':'float64', 'dispuniform':'float16', 'dmonth':'float16', 'dqtr':'float16', 'drg':'float16', 'drgver':'float16', 'drg_nopoa':'float16', 'i10_dx1':'string', 'i10_dx2':'string', 'i10_dx3':'string', 'i10_dx4':'string', 'i10_dx5':'string', 'i10_dx6':'string', 'i10_dx7':'string', 'i10_dx8':'string', 'i10_dx9':'string', 'i10_dx10':'string', 'i10_dx11':'string', 'i10_dx12':'string', 'i10_dx13':'string', 'i10_dx14':'string', 'i10_dx15':'string', 'i10_dx16':'string', 'i10_dx17':'string', 'i10_dx18':'string', 'i10_dx19':'string', 'i10_dx20':'string', 'i10_dx21':'string', 'i10_dx22':'string','i10_dx23':'string','i10_dx24':'string', 'i10_dx25':'string','i10_dx26':'string','i10_dx27':'string','i10_dx28':'string','i10_dx29':'string', 'i10_dx30':'string', 'i10_dx31':'string', 'i10_dx32':'string', 'i10_dx33':'string', 'i10_dx34':'string', 'i10_dx35':'string', 'i10_dx36':'string', 'i10_dx37':'string', 'i10_dx38':'string', 'i10_dx39':'string', 'i10_dx40':'string', 'elective':'float16', 'female':'float16', 'hcup_ed':'float16', 'hosp_nrd':'float64', 'los': 'float16', 'mdc':'float16', 'mdc_nopoa':'float16', 'i10_ndx':'float16', 'i10_npr':'float16', 'nrd_daystoevent':'float16', 'nrd_stratum':'float16', 'nrd_visitlink':'string', 'pay1':'float16', 'pl_nchs':'float16', 'i10_pr1':'string', 'i10_pr2':'string', 'i10_pr3':'string', 'i10_pr4':'string', 'i10_pr5':'string', 'i10_pr6':'string', 'i10_pr7':'string', 'i10_pr8':'string', 'i10_pr9':'string', 'i10_pr10':'string', 'i10_pr11':'string', 'i10_pr12':'string', 'i10_pr13':'string', 'i10_pr14':'string', 'i10_pr15':'string', 'i10_pr16':'string', 'i10_pr17':'string', 'i10_pr18':'string', 'i10_pr19':'string', 'i10_pr20':'string', 'i10_pr21':'string', 'i10_pr22':'string', 'i10_pr23':'string', 'i10_pr24':'string', 'i10_pr25':'string', 'prday1':'float16', 'prday2':'float16', 'prday3':'float16', 'prday4':'float16', 'prday5':'float16', 'prday6':'float16', 'prday7':'float16', 'prday8':'float16', 'prday9':'float16', 'prday10':'float16', 'prday11':'float16', 'prday12':'float16', 'prday13':'float16', 'prday14':'float16', 'prday15':'float16', 'prday16':'float16', 'prday17':'float16', 'prday18':'float16', 'prday19':'float16', 'prday20':'float16', 'prday21':'float16', 'prday22':'float16', 'prday23':'float16', 'prday24':'float16', 'prday25':'float16', 'rehabtransfer':'float16', 'resident':'float16', 'samedayevent':'float16', 'totchg':'float64', 'year':'float16', 'zipinc_qrtl':'float16', 'dxver':'float16', 'prver':'float16', 'preventable_readmission':'float16', 'prev_readm_reason':'float16', 'medical_admission':'float16', 'hfrs_score':'float16', 'hfrs_severity':'float16', 'cc_score':'float16', 'cc_severity':'float16'})

In [3]:
# Take a quick look at the file structure.
core_file.head()

,key_nrd,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd_x,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum_x,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year_x,zipinc_qrtl,dxver,prver,hosp_bedsize,h_contrl,hosp_urcat4,hosp_ur_teach,nrd_stratum_y,n_disc_u,n_hosp_u,s_disc_u,s_hosp_u,total_disc,year_y,aprdrg,aprdrg_risk_mortality,aprdrg_severity,hosp_nrd_y,next_nrdday,days_to_readmission,index_admission,readmission_diagnosis,readmission_los,preventable_admission,pqi_1,pqi_3,pqi_5,pqi_7,pqi_8,pqi_11,pqi_12,pqi_14,pqi_15,pqi_16,pqi_5_15_exclusion,pqi_7_medical_exclusions,pqi_7_procedural_exclusion,pqi_7_8_procedural_exclusion,pqi_11_exclusion,pqi_12_exclusion,pqi_immunocompromise_medical_exclusion,pqi_immunocompromise_procedural_exclusion,pqi_16_exclusion_procedure,pqi_16_amputation_procedure,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,cc_MI,cc_CHF,cc_PVD,cc_CVD,cc_dementia,cc_COPD,cc_rheum,cc_peptic_ulcer,cc_mild_liver_disease,cc_diabetes_wo_complications,cc_diabetes_w_complications,cc_plegia,cc_renal,cc_malignancy,cc_severe_liver,cc_metastasis,cc_AIDS
0,2.017131e+11,90.0,0.0,0.0,1.763376,5.0,1.0,1.0,641.0,34.0,641.0,E876,I10,I4891,D649,K219,E039,G309,F0280,F329,M1990,E440,I252,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,1.0,0.0,43871.0,4.0,10.0,10.0,12.0,0.0,15888.0,458.0,e000029,1.0,3.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,6142.0,2017.0,3.0,10.0,10.0,1.0,2.0,2.0,1.0,458,283276,40,141746,26,3527,2017,425.0,2.0,2.0,43871.0,NaN,NaN,1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.703125,1.0,2.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2.017084e+11,44.0,0.0,0.0,1.940491,1.0,9.0,3.0,989.0,34.0,989.0,S0269XA,S0266XA,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,1.0,1.0,42425.0,0.0,3.0,3.0,2.0,1.0,19088.0,435.0,e00008o,3.0,2.0,0JQ10ZZ,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,4211.0,2017.0,2.0,10.0,10.0,3.0,2.0,1.0,1.0,435,2849102,73,1360786,41,19512,2017,952.0,1.0,1.0,42425.0,NaN,NaN,1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2.017165e+11,18.0,0.0,0.0,2.032179,6.0,4.0,2.0,765.0,34.0,765.0,O6014X0,A408,F329,O9081,F419,N12,Z87891,O76,O99344,D62,Z3A35,O753,Z370,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.0,1.0,0.0,42717.0,3.0,14.0,14.0,13.0,2.0,21264.0,421.0,e00009m,2.0,3.0,10907ZC,10D00Z1,<NA>,<NA>,<NA>,<

## Capturing only patients who have IBD related diagnoses

In [4]:
# Create a list of column names
column_list = core_file.columns.to_list()

# This dictionary holds the fill in values for any blanks or NaN's in the data for the columns beginning with 'i10_dx'.
# If there is no discharge diagnosis, it will be filled in with a value of '' so we can iterate through the rows without any errors below for NaN's.
fill_values = {}

# i10_dx_columns will hold the column's numerical position, example: i10_dx1 is the 10th column (starting from 0).
# This will be used later to loop through the diagnosis columns specifically.
i10_dx_columns = []

# copy_columns will hold a list of columns that begin with 'i10_dx', initiate with nrd_visitlink since we need that as well.
copy_columns = ['nrd_visitlink']

# counter initialized at 2 because we have 2 items already in the copy_columns list.
counter = 1

# This loop goes through the columns in the core_file dataframe, finds the columns beginning with 'i10_dx', adds them to the dictionary and counts their position.
for column in column_list:
    if 'i10_dx' in column:
        fill_values.update({column: ''})
        i10_dx_columns.append(counter)
        copy_columns.append(column)
        counter += 1

# Create a copy of the original dataframe, including only specified columns, fill in NaN with ''.
i10_dx_df = core_file[copy_columns].copy().fillna(value = fill_values)

In [5]:
# A list of patients who have an ibd diagnosis in any of the 'i10_dx' columns. ibd diagnosis codes begin with 'K50' or 'K51'.
patient_list = []
patient_list = [row.nrd_visitlink for row in tqdm(i10_dx_df.itertuples(index=False)) for position in i10_dx_columns if 'K50' in row[position] or 'K51' in row[position]]

# Unique list of patients from the list above.
unique_patients = np.unique(patient_list)

16647521it [09:39, 28740.62it/s]


In [6]:
# A dictionary that will map a 1 for every row that has a nrd_visitlink in the unique_patients list, this allows us to quickly create a new dataframe with only the cohort required.
unique_patients_dict = {value:1 for value in unique_patients}

# Create a new column called in_cohort by mapping the nrd_visitlinks values from the dicionary unique_patients_dict.
core_file["in_cohort"] = core_file["nrd_visitlink"].map(unique_patients_dict)

# Use masking to create a dataframe which only has the patients that we are looking for, where in_cohort == 1.
mask = core_file["in_cohort"].values == 1
cohort_file = core_file.loc[mask]

# Drop the unnecessary column and reset the index.
cohort_file = cohort_file.drop(columns=['in_cohort']).reset_index(drop=True)

In [7]:
# Turn the dask dataframe into a pandas dataframe now that the heavy lifting has been completed. This should only take a few minutes depending on file size.
cohort_file = cohort_file.compute(index=False, single_file=True)

# Reset the index and drop the old column since dask works in partitions and will have weird index numbers.
cohort_file = cohort_file.reset_index(drop=True)

# Sort the file in ascending order based on the values of the 'nrd_visitlink' column so all of the patient visits are grouped together.
# Secondly, sort by 'dmonth' so that the patients are sorted from January through December.
cohort_file = cohort_file.sort_values(by=['nrd_visitlink', 'dmonth']).reset_index(drop=True)

cohort_file

,key_nrd,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd_x,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum_x,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year_x,zipinc_qrtl,dxver,prver,hosp_bedsize,h_contrl,hosp_urcat4,hosp_ur_teach,nrd_stratum_y,n_disc_u,n_hosp_u,s_disc_u,s_hosp_u,total_disc,year_y,aprdrg,aprdrg_risk_mortality,aprdrg_severity,hosp_nrd_y,next_nrdday,days_to_readmission,index_admission,readmission_diagnosis,readmission_los,preventable_admission,pqi_1,pqi_3,pqi_5,pqi_7,pqi_8,pqi_11,pqi_12,pqi_14,pqi_15,pqi_16,pqi_5_15_exclusion,pqi_7_medical_exclusions,pqi_7_procedural_exclusion,pqi_7_8_procedural_exclusion,pqi_11_exclusion,pqi_12_exclusion,pqi_immunocompromise_medical_exclusion,pqi_immunocompromise_procedural_exclusion,pqi_16_exclusion_procedure,pqi_16_amputation_procedure,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,cc_MI,cc_CHF,cc_PVD,cc_CVD,cc_dementia,cc_COPD,cc_rheum,cc_peptic_ulcer,cc_mild_liver_disease,cc_diabetes_wo_complications,cc_diabetes_w_complications,cc_plegia,cc_renal,cc_malignancy,cc_severe_liver,cc_metastasis,cc_AIDS
0,2.017114e+11,88.0,0.0,0.0,1.805447,1.0,5.0,2.0,470.0,34.0,470.0,M1712,I10,L309,K5190,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.0,0.0,0.0,42214.0,1.0,8.0,8.0,4.0,1.0,13976.0,447.0,e0007w8,1.0,3.0,0SRD0J9,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,43848.0,2017.0,2.0,10.0,10.0,3.0,1.0,2.0,1.0,447,636191,24,332444,15,21737,2017,302.0,1.0,2.0,42214.0,NaN,NaN,1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2.017057e+11,49.0,1.0,0.0,1.492757,1.0,6.0,2.0,386.0,34.0,386.0,K50812,G40909,J449,F17210,F329,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,0.0,1.0,41827.0,3.0,6.0,6.0,5.0,0.0,19296.0,452.0,e0009vi,2.0,6.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,11700.0,2017.0,2.0,10.0,10.0,2.0,2.0,4.0,2.0,452,18409,9,11959,7,2872,2017,245.0,1.0,1.0,41827.0,NaN,NaN,1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.000000,0.0,1.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2.017025e+11,51.0,0.0,0.0,1.631724,7.0,1.0,1.0,872.0,34.0,872.0,A419,I10,Z5321,A09,G8929,Z9884,E669,D62,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,1.0,1.0,44540.0,1.0,18.0,18.0,8.0,0.0,13632.0,453.0,e000fgg,1.0,5.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

## Keep only adults in the cohort

In [8]:
# Now we need to carve out the cohort by removing any patients who are younger than 18 and do not have an IBD diagnosis between January - June 2017 (the first 6 months).

# These commented out methods do not work because there are some patients who were 17 initially but turned 18 at a later diagnosis. We are removing those patients as they were not
# adults when first receiving a diagnosis.
# ibd_file = pd.concat([pd.DataFrame([row],columns = row._fields) for row in tqdm(cohort_file.itertuples(index=False)) if row.age > 17]) # slower than using .loc
# ibd_file = cohort_file.loc[cohort_file['age'] > 17]

# First we'll create a list of patients who are not adults (ages 17 and under) and filter it for unique nrd_visitlink ID's.
# This list automatically removes any patients who may have turned 18 after their initial diagnosis when they were 17.
not_adult = [row.nrd_visitlink for row in tqdm(cohort_file.itertuples(index=False)) if row.age < 18]
not_adult_unique = np.unique(not_adult)

# Create a list of patients from the dataframe that are not in the list of patients who are ages 17 and under and filter for unique nrd_visitlinks.
adult_patients = [row.nrd_visitlink for row in tqdm(cohort_file.itertuples(index=False)) if row.nrd_visitlink not in not_adult_unique]
adult_patients_unique = np.unique(adult_patients)

print(f'\nNumber of unique patients in the cohort: {len(adult_patients_unique):,}')

# The next two lines were the original method used to create the dataframe. It is significanly slower and can take up to 20+ minutes to run, but it does work.
# Then we will create a new dataframe (defined_cohort_file) with nrd_visitlink's not in the list, therefore creating a cohort of only adults (ages 18+).
# defined_cohort_file = pd.concat([pd.DataFrame([row],columns = row._fields) for row in tqdm(cohort_file.itertuples(index=False)) if row.nrd_visitlink not in not_adult_unique])

# Then we will create a new dataframe (defined_cohort_file) with nrd_visitlink's from the adult_patient list.
# "|".join(all_patients_unique) creates a single string value with a "|" as a separator, allowing the .contains() method to search for each unique patient nrd_visitlink.
# defined_cohort_file = cohort_file.loc[cohort_file['nrd_visitlink'].str.contains("|".join(adult_patients_unique))]



# A dictionary that will map a 1 for every row that has a nrd_visitlink in the adult_patients_unique list, this allows us to quickly create a new dataframe with only the cohort
# required.
unique_adults_dict = {value:1 for value in adult_patients_unique}

# Create a new column called in_cohort by mapping the nrd_visitlinks values from the dicionary unique_adults_dict.
cohort_file["in_cohort"] = cohort_file["nrd_visitlink"].map(unique_adults_dict)

# Use masking to create a dataframe which only has the patients that we are looking for, where in_cohort == 1.
masking = cohort_file["in_cohort"].values == 1
defined_cohort_file = cohort_file.loc[masking]

# Drop the unnecessary column and reset the index.
defined_cohort_file.drop(columns=['in_cohort']).reset_index(drop=True)

228714it [00:07, 28991.19it/s]
228714it [00:10, 21396.28it/s]

Number of unique patients in the cohort: 127,596


,key_nrd,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd_x,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum_x,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year_x,zipinc_qrtl,dxver,prver,hosp_bedsize,h_contrl,hosp_urcat4,hosp_ur_teach,nrd_stratum_y,n_disc_u,n_hosp_u,s_disc_u,s_hosp_u,total_disc,year_y,aprdrg,aprdrg_risk_mortality,aprdrg_severity,hosp_nrd_y,next_nrdday,days_to_readmission,index_admission,readmission_diagnosis,readmission_los,preventable_admission,pqi_1,pqi_3,pqi_5,pqi_7,pqi_8,pqi_11,pqi_12,pqi_14,pqi_15,pqi_16,pqi_5_15_exclusion,pqi_7_medical_exclusions,pqi_7_procedural_exclusion,pqi_7_8_procedural_exclusion,pqi_11_exclusion,pqi_12_exclusion,pqi_immunocompromise_medical_exclusion,pqi_immunocompromise_procedural_exclusion,pqi_16_exclusion_procedure,pqi_16_amputation_procedure,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,cc_MI,cc_CHF,cc_PVD,cc_CVD,cc_dementia,cc_COPD,cc_rheum,cc_peptic_ulcer,cc_mild_liver_disease,cc_diabetes_wo_complications,cc_diabetes_w_complications,cc_plegia,cc_renal,cc_malignancy,cc_severe_liver,cc_metastasis,cc_AIDS
0,2.017114e+11,88.0,0.0,0.0,1.805447,1.0,5.0,2.0,470.0,34.0,470.0,M1712,I10,L309,K5190,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.0,0.0,0.0,42214.0,1.0,8.0,8.0,4.0,1.0,13976.0,447.0,e0007w8,1.0,3.0,0SRD0J9,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,43848.0,2017.0,2.0,10.0,10.0,3.0,1.0,2.0,1.0,447,636191,24,332444,15,21737,2017,302.0,1.0,2.0,42214.0,NaN,NaN,1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2.017057e+11,49.0,1.0,0.0,1.492757,1.0,6.0,2.0,386.0,34.0,386.0,K50812,G40909,J449,F17210,F329,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,0.0,1.0,41827.0,3.0,6.0,6.0,5.0,0.0,19296.0,452.0,e0009vi,2.0,6.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,11700.0,2017.0,2.0,10.0,10.0,2.0,2.0,4.0,2.0,452,18409,9,11959,7,2872,2017,245.0,1.0,1.0,41827.0,NaN,NaN,1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.000000,0.0,1.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2.017025e+11,51.0,0.0,0.0,1.631724,7.0,1.0,1.0,872.0,34.0,872.0,A419,I10,Z5321,A09,G8929,Z9884,E669,D62,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,1.0,1.0,44540.0,1.0,18.0,18.0,8.0,0.0,13632.0,453.0,e000fgg,1.0,5.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [9]:
# Reset the index in case we need to reference it later.
defined_cohort_file = defined_cohort_file.reset_index(drop=True)

## Find all patients who had an IBD related discharge in the first 6 months

In [10]:
# Create a list of column names.
new_column_list = defined_cohort_file.columns.to_list()

# This dictionary holds the fill in values for any blanks or NaN's in the data for the columns beginning with 'i10_dx'.
# If there is no discharge diagnosis, it will be filled in with a value of '' so we can iterate through the rows without any errors below for NaN's.
new_fill_values = {}

# i10_dx_columns will hold the column's numerical position, example: i10_dx1 is the 10th column (starting from 0).
# This will be used later to loop through the diagnosis columns specifically.
new_i10_dx_columns = []

# copy_columns will hold a list of columns that begin with 'i10_dx', initiate with dmonth and nrd_visitlink since we need those as well.
new_copy_columns = ['dmonth', 'nrd_visitlink']

# counter initialized at 2 because we have 2 items already in the copy_columns list.
new_counter = 2

# This loop goes through the columns in the defined_cohort_file dataframe, finds the columns beginning with 'i10_dx', adds them to the dictionary and counts their position.
for column in new_column_list:
    if 'i10_dx' in column:
        new_fill_values.update({column: ''})
        new_i10_dx_columns.append(new_counter)
        new_copy_columns.append(column)
        new_counter += 1

# Create a copy of the original dataframe, including only specified columns, fill in NaN with ''.
new_i10_dx_df = defined_cohort_file[new_copy_columns].copy().fillna(value = new_fill_values)

In [11]:
# Create a list of patient ID's who have an IBD related index hospitalization between January - June 2017 (the first 6 months).
new_patient_list = []
new_patient_list = [row.nrd_visitlink for row in tqdm(new_i10_dx_df.itertuples(index=False)) for position in new_i10_dx_columns if row.dmonth < 7 and 'K50' in row[position] or row.dmonth < 7 and 'K51' in row[position]]

new_unique_patients = np.unique(new_patient_list)

228714it [00:04, 48491.70it/s]


In [12]:
# Go through the dataset and use concat to append any rows that have nrd_visitlinks in the unique_patients list to final_cohort_file (a new dataframe) because these are the patients we want to look at.

# Similar to above, this method is far slower and can take 15+ minutes to run even though it works.
# final_cohort_file = pd.concat([pd.DataFrame([row],columns = row._fields) for row in tqdm(defined_cohort_file.itertuples(index=False)) if row.nrd_visitlink in unique_patients])

# Using .loc is much faster and works without having to compute dask.
# final_cohort_file = defined_cohort_file.loc[defined_cohort_file['nrd_visitlink'].str.contains("|".join(unique_patients))]


# A dictionary that will map a 1 for every row that has a nrd_visitlink in the new_unique_patients list, this allows us to quickly create a new dataframe with only the cohort
# required.
unique_final_dict = {value:1 for value in new_unique_patients}

# Create a new column called in_cohort by mapping the nrd_visitlinks values from the dicionary unique_adults_dict.
defined_cohort_file["in_cohort"] = defined_cohort_file["nrd_visitlink"].map(unique_final_dict)

# Use masking to create a dataframe which only has the patients that we are looking for, where in_cohort == 1.
masking = defined_cohort_file["in_cohort"].values == 1
final_cohort_file = defined_cohort_file.loc[masking]

# Drop the unnecessary column and reset the index.
final_cohort_file = final_cohort_file.drop(columns=['in_cohort']).reset_index(drop=True)

# Sort the file in ascending order based on the values of the 'nrd_visitlink' column so all of the patient visits are grouped together.
# Secondly, sort by 'dmonth' so that the patients are sorted from January through December.
final_cohort_file = final_cohort_file.sort_values(by=['nrd_visitlink', 'dmonth']).reset_index(drop=True)

final_cohort_file

,key_nrd,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd_x,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum_x,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year_x,zipinc_qrtl,dxver,prver,hosp_bedsize,h_contrl,hosp_urcat4,hosp_ur_teach,nrd_stratum_y,n_disc_u,n_hosp_u,s_disc_u,s_hosp_u,total_disc,year_y,aprdrg,aprdrg_risk_mortality,aprdrg_severity,hosp_nrd_y,next_nrdday,days_to_readmission,index_admission,readmission_diagnosis,readmission_los,preventable_admission,pqi_1,pqi_3,pqi_5,pqi_7,pqi_8,pqi_11,pqi_12,pqi_14,pqi_15,pqi_16,pqi_5_15_exclusion,pqi_7_medical_exclusions,pqi_7_procedural_exclusion,pqi_7_8_procedural_exclusion,pqi_11_exclusion,pqi_12_exclusion,pqi_immunocompromise_medical_exclusion,pqi_immunocompromise_procedural_exclusion,pqi_16_exclusion_procedure,pqi_16_amputation_procedure,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,cc_MI,cc_CHF,cc_PVD,cc_CVD,cc_dementia,cc_COPD,cc_rheum,cc_peptic_ulcer,cc_mild_liver_disease,cc_diabetes_wo_complications,cc_diabetes_w_complications,cc_plegia,cc_renal,cc_malignancy,cc_severe_liver,cc_metastasis,cc_AIDS
0,2.017114e+11,88.0,0.0,0.0,1.805447,1.0,5.0,2.0,470.0,34.0,470.0,M1712,I10,L309,K5190,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.0,0.0,0.0,42214.0,1.0,8.0,8.0,4.0,1.0,13976.0,447.0,e0007w8,1.0,3.0,0SRD0J9,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,43848.0,2017.0,2.0,10.0,10.0,3.0,1.0,2.0,1.0,447,636191,24,332444,15,21737,2017,302.0,1.0,2.0,42214.0,NaN,NaN,1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2.017057e+11,49.0,1.0,0.0,1.492757,1.0,6.0,2.0,386.0,34.0,386.0,K50812,G40909,J449,F17210,F329,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,0.0,1.0,41827.0,3.0,6.0,6.0,5.0,0.0,19296.0,452.0,e0009vi,2.0,6.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,11700.0,2017.0,2.0,10.0,10.0,2.0,2.0,4.0,2.0,452,18409,9,11959,7,2872,2017,245.0,1.0,1.0,41827.0,NaN,NaN,1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.000000,0.0,1.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2.017025e+11,51.0,0.0,0.0,1.631724,7.0,1.0,1.0,872.0,34.0,872.0,A419,I10,Z5321,A09,G8929,Z9884,E669,D62,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,1.0,1.0,44540.0,1.0,18.0,18.0,8.0,0.0,13632.0,453.0,e000fgg,1.0,5.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [13]:
print(f'Total number of patients in the cohort: {len(new_patient_list):,}') #94,389

# A new list of unique patient ID's from patient_list.
print(f'\nNumber of unique patients in the cohort: {len(new_unique_patients):,}') #70,483

# How many hospital visits are there?
print(f'\nTotal number of hospital visits spread across all patients in the cohort: {len(final_cohort_file):,}') #139,265

Total number of patients in the cohort: 94,405

Number of unique patients in the cohort: 70,495

Total number of hospital visits spread across all patients in the cohort: 139,286


## ibd_diagnosis and ibd_index_hospitalization columns

In [14]:
# Create a new column called 'ibd_diagnosis' and initialize with a 0 to start.
final_cohort_file['ibd_diagnosis'] = 0

# Create a list of column names that include i10_dx.
diagnoses = [column for column in column_list if 'i10_dx' in column]

# Initialize a variable at 1 to correspond with the first i10_dx column.
x = 1

# For each column in the diagnoses list (i10_dx1 - i10_dx40), create a new column with the corresponding discharge number (ex: index1, index2, etc) and fill in a 1 if they had an
# IBD related diagnosis in that column (ex: if K50 in i10_dx3 then fill in a 1 for that row in index3).
for column in diagnoses:
    final_cohort_file[f'index{x}'] = 0
    final_cohort_file.loc[final_cohort_file[column].str.contains('K50|K51'), f'index{x}'] = 1
    x += 1

In [15]:
# Create a list of columns from the dataframe.
index_columns = final_cohort_file.columns.to_list()

# Initialize a list to hold the numerical position of the columns and a counter.
index_column_positions = []
y = 0

# Create a list of columns only if they include 'index' in their name (these are the new columns we just created).
for column in index_columns:
    if 'index' in column:
        index_column_positions.append(y)
    y += 1

# ibd_counts is a new column that sums up the values in all of the 'index' columns so we can create the ibd_diagnosis column.
final_cohort_file['ibd_counts'] = final_cohort_file.iloc[:, index_column_positions[0]:index_column_positions[-1]+1].sum(axis=1)

In [16]:
# Making sure all the created columns exist.
final_cohort_file

,key_nrd,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd_x,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum_x,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year_x,zipinc_qrtl,dxver,prver,hosp_bedsize,h_contrl,hosp_urcat4,hosp_ur_teach,nrd_stratum_y,n_disc_u,n_hosp_u,s_disc_u,s_hosp_u,total_disc,year_y,aprdrg,aprdrg_risk_mortality,aprdrg_severity,hosp_nrd_y,next_nrdday,days_to_readmission,index_admission,readmission_diagnosis,readmission_los,preventable_admission,pqi_1,pqi_3,pqi_5,pqi_7,pqi_8,pqi_11,pqi_12,pqi_14,pqi_15,pqi_16,pqi_5_15_exclusion,pqi_7_medical_exclusions,pqi_7_procedural_exclusion,pqi_7_8_procedural_exclusion,pqi_11_exclusion,pqi_12_exclusion,pqi_immunocompromise_medical_exclusion,pqi_immunocompromise_procedural_exclusion,pqi_16_exclusion_procedure,pqi_16_amputation_procedure,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,cc_MI,cc_CHF,cc_PVD,cc_CVD,cc_dementia,cc_COPD,cc_rheum,cc_peptic_ulcer,cc_mild_liver_disease,cc_diabetes_wo_complications,cc_diabetes_w_complications,cc_plegia,cc_renal,cc_malignancy,cc_severe_liver,cc_metastasis,cc_AIDS,ibd_diagnosis,index1,index2,index3,index4,index5,index6,index7,index8,index9,index10,index11,index12,index13,index14,index15,index16,index17,index18,index19,index20,index21,index22,index23,index24,index25,index26,index27,index28,index29,index30,index31,index32,index33,index34,index35,index36,index37,index38,index39,index40,ibd_counts
0,2.017114e+11,88.0,0.0,0.0,1.805447,1.0,5.0,2.0,470.0,34.0,470.0,M1712,I10,L309,K5190,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.0,0.0,0.0,42214.0,1.0,8.0,8.0,4.0,1.0,13976.0,447.0,e0007w8,1.0,3.0,0SRD0J9,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,43848.0,2017.0,2.0,10.0,10.0,3.0,1.0,2.0,1.0,447,636191,24,332444,15,21737,2017,302.0,1.0,2.0,42214.0,NaN,NaN,1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.000000
1,2.017057e+11,49.0,1.0,0.0,1.492757,1.0,6.0,2.0,386.0,34.0,386.0,K50812,G40909,J449,F17210,F329,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,0.0,1.0,41827.0,3.0,6.0,6.0,5.0,0.0,19296.0,452.0,e0009vi,2.0,6.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,11700.0,2017.0,2.0,10.0,10.0,2.0,2.0,4.0,2.0,452,18409,9,11959,7,2872,2017,245.0,1.0,1.0,41827.0,NaN,NaN,1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.000000,0.0,

In [17]:
# Start with a placeholder for nrd_visitlink that is made up ('aaaaaa'). This will hold the previous rows nrd_visitlink in each iteration.
# Keep a counter for each successive ibd_diagnosis that a patient has, initialized with 1.
# row_num keeps track of which row we are on.
id_placeholder = 'aaaaaa'
counter = 1
row_num = 0

# Initialize the 'ibd_diagnosis' column with a 0.
final_cohort_file['ibd_diagnosis'] = 0

# This loop goes through each row and compares the nrd_visitlink with that of the previous row. 
for row in tqdm(final_cohort_file.itertuples(index=False)):
    id_current = row.nrd_visitlink
    # If the current and previous nrd_visitlink are not the same and the patient has a ibd related discharge (ibd_counts >= 1), then a 1 is populated in ibd_diagnosis. Counter reset
    # to 1.
    if id_current != id_placeholder and row.ibd_counts >= 1:
        final_cohort_file.at[row_num, 'ibd_diagnosis'] = 1
        id_placeholder = id_current
        counter = 1
    # If the current and previous nrd_visitlink are the same and the patient has a idb related discharge (ibd_counts >= 1), then the counter increments by 1 and the appropriate
    # number is populated in ibd_diagnosis (each subsequent ibd discharge adds 1 to the patients counter).
    elif id_current == id_placeholder and row.ibd_counts >= 1:
        counter += 1
        final_cohort_file.at[row_num, 'ibd_diagnosis'] = counter
        id_placeholder = id_current
    # If the current and previous nrd_visitlink are the same and the patient does not have an ibd related discharge (ibd_counts < 1), then a 0 is populated in ibd_diagnosis.
    elif id_current == id_placeholder and row.ibd_counts < 1:
        final_cohort_file.at[row_num, 'ibd_diagnosis'] = 0
        id_placeholder = id_current
    # If the current and previous nrd_visitlink are not the same and the patient does not have an idb related discharge, then a 0 is populated in ibd_diagnosis. Counter reset 
    # to 0. id_placeholder is updated in all cases to the current nrd_visitlink.
    else:
        final_cohort_file.at[row_num, 'ibd_diagnosis'] = 0
        id_placeholder = id_current
        counter = 0
    row_num += 1

139286it [00:06, 20331.83it/s]


In [18]:
# Remove all the unnecessary columns we created that begin with 'index'.
drop = final_cohort_file.columns.to_list()
drop_columns = [column for column in drop if 'index' in column]
for column in drop_columns:
    final_cohort_file = final_cohort_file.drop(columns=[column])

# Drop the 'ibd_counts' column since we no longer need it either.
final_cohort_file = final_cohort_file.drop(columns=['ibd_counts'])

In [19]:
# Create a new column called 'ibd_index_hospitalization' and fill each row with a 0.
# Use vectorization with masking to find each index hospitalization and fill in a 1 in ibd_index_hospitalization.
final_cohort_file['ibd_index_hospitalization'] = 0
mask = (final_cohort_file['ibd_diagnosis'] == 1)
final_cohort_file.loc[mask, 'ibd_index_hospitalization'] = 1

In [20]:
# Check the file structure.
final_cohort_file

,key_nrd,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd_x,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum_x,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year_x,zipinc_qrtl,dxver,prver,hosp_bedsize,h_contrl,hosp_urcat4,hosp_ur_teach,nrd_stratum_y,n_disc_u,n_hosp_u,s_disc_u,s_hosp_u,total_disc,year_y,aprdrg,aprdrg_risk_mortality,aprdrg_severity,hosp_nrd_y,next_nrdday,days_to_readmission,readmission_diagnosis,readmission_los,preventable_admission,pqi_1,pqi_3,pqi_5,pqi_7,pqi_8,pqi_11,pqi_12,pqi_14,pqi_15,pqi_16,pqi_5_15_exclusion,pqi_7_medical_exclusions,pqi_7_procedural_exclusion,pqi_7_8_procedural_exclusion,pqi_11_exclusion,pqi_12_exclusion,pqi_immunocompromise_medical_exclusion,pqi_immunocompromise_procedural_exclusion,pqi_16_exclusion_procedure,pqi_16_amputation_procedure,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,cc_MI,cc_CHF,cc_PVD,cc_CVD,cc_dementia,cc_COPD,cc_rheum,cc_peptic_ulcer,cc_mild_liver_disease,cc_diabetes_wo_complications,cc_diabetes_w_complications,cc_plegia,cc_renal,cc_malignancy,cc_severe_liver,cc_metastasis,cc_AIDS,ibd_diagnosis,ibd_index_hospitalization
0,2.017114e+11,88.0,0.0,0.0,1.805447,1.0,5.0,2.0,470.0,34.0,470.0,M1712,I10,L309,K5190,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.0,0.0,0.0,42214.0,1.0,8.0,8.0,4.0,1.0,13976.0,447.0,e0007w8,1.0,3.0,0SRD0J9,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,43848.0,2017.0,2.0,10.0,10.0,3.0,1.0,2.0,1.0,447,636191,24,332444,15,21737,2017,302.0,1.0,2.0,42214.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,2.017057e+11,49.0,1.0,0.0,1.492757,1.0,6.0,2.0,386.0,34.0,386.0,K50812,G40909,J449,F17210,F329,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,0.0,1.0,41827.0,3.0,6.0,6.0,5.0,0.0,19296.0,452.0,e0009vi,2.0,6.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,11700.0,2017.0,2.0,10.0,10.0,2.0,2.0,4.0,2.0,452,18409,9,11959,7,2872,2017,245.0,1.0,1.0,41827.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.000000,0.0,1.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
2,2.017025e+11,51.0,0.0,0.0,1.631724,7.0,1.0,1.0,872.0,34.0,872.0,A419,I10,Z5321,A09,G8929,Z9884,E669,D62,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,1.0,1.0,44540.0,1.0,18.0,18.0,8.0,0.0,13632.0,453.0,e000fgg,1.0,5.0,<NA>,<NA>,<NA>,<NA>

## These will be brought in on the original merge
### key_nrd and other variables from hospital and severity files

In [21]:
# # Reading in the core file to capture 'key_nrd' as it was dropped on the original merge at the very end. Sort it the same way as final_cohort_file is sorted.
# core_file = pd.read_csv('NRD_2017_Core.csv', usecols=['dmonth', 'key_nrd', 'nrd_visitlink'], dtype={'dmonth':'float16', 'key_nrd':'float64', 'nrd_visitlink':str}).sort_values(by=['nrd_visitlink', 'dmonth']).reset_index(drop=True)

# # Only keep the patients who are in the list of unique patient nrd_visitlink's from above.
# core_file = core_file[core_file['nrd_visitlink'].isin(unique_patients)]
# core_file = core_file.reset_index(drop=True)

# # Create a dictionary with nrd_visitlink's and their respective key_nrd values and map it to the new 'key_nrd' column in final_cohort_file.
# mapping = dict(core_file[['nrd_visitlink', 'key_nrd']].values)
# final_cohort_file['key_nrd'] = final_cohort_file['nrd_visitlink'].map(mapping)

# print('key_nrd mapping complete')

In [22]:
# # Reading in the severity and hospital files as well and bringing in variables that were not captured but are part of the variable list.
# hospital_2017 = pd.read_csv('NRD_2017_Hospital.csv', usecols=['hosp_bedsize', 'h_contrl', 'hosp_nrd', 'hosp_urcat4', 'hosp_ur_teach', 'n_disc_u', 'n_hosp_u', 's_disc_u', 's_hosp_u', 'total_disc'], dtype={'hosp_bedsize':'float16', 'h_contrl':'float16', 'hosp_nrd':'float64', 'hosp_urcat4':'float16', 'hosp_ur_teach':'float16', 'n_disc_u':'float64', 'n_hosp_u':'float16', 's_disc_u':'float64', 's_hosp_u':'float16', 'total_disc':'float64'})

# severity_2017 = pd.read_csv('NRD_2017_Severity.csv', dtype={'aprdrg':'float16', 'aprdrg_risk_mortality':'float16', 'aprdrg_severity':'float16', 'hosp_nrd':'float64', 'key_nrd':'float64'})

# final_cohort_file = pd.merge(final_cohort_file, hospital_2017, how='left', on='hosp_nrd')
# print('hospital merge complete')

# final_cohort_file = pd.merge(final_cohort_file, severity_2017, how='left', on='key_nrd')
# final_cohort_file = final_cohort_file.drop(columns=['hosp_nrd_y'])
# final_cohort_file = final_cohort_file.rename(columns={'hosp_nrd_x':'hosp_nrd'})
# print('severity merge complete')

## Creating additional columns

In [23]:
# Create a list of diagnosis columns, all columns that begin with 'i10_dx'.
diagnoses = [column for column in column_list if 'i10_dx' in column]

# Create a list of procedure columns, all columns that begin with 'i10_pr'.
procedures = [column for column in column_list if 'i10_pr' in column]

In [24]:
# Define a function that will take in the following:
# - df = the dataframe the columns are located in.
# - list_of_columns = a list of column names that will be used to find diagnosis/procedure codes from the dataframe.
# - column_label = a string value that will be used to initialize and create a new column that stores either a 1 or 0 depending on if the patient has a specific diagnosis/procedure.
# - search_codes = a single string value with the starting values for ICD-10 codes for the diagnosis/procedure that is being located.
#   - The format for search_codes should be as follows: 'code|code|code|code'; using the | separator allows individual searches and 'code' can be a partial code such as F80 instead
#     of every possible outcome like F801, F8009, F80901, etc.

# This function will create a new column with the desired name and populate it with a 1 or a 0 depending on if a patient has a specific diagnosis/procedure on that discharge.
def icd10_to_column(df, list_of_columns, column_label, search_codes):
    df[column_label] = 0
    for column in list_of_columns:
        df.loc[df[column].str.contains(search_codes), column_label] = 1
    return print(f'Column for {column_label} created.')

In [25]:
# This dictionary holds the fill in values for any blanks or NaN's in the data for the columns beginning with 'i10_pr'.
# If there is no procedure, it will be filled in with a value of '' so we can iterate through the rows without any errors below for NaN's.
fill_values_pr = {}

# copy_columns will hold a list of columns that begin with 'i10_pr'
copy_columns_pr = []

# This loop goes through the columns in the final_cohort_file dataframe, finds the columns beginning with 'i10_pr', adds them to the dictionary.
for column in column_list:
    if 'i10_pr' in column:
        fill_values_pr.update({column: ''})
        copy_columns_pr.append(column)

# Create a copy of the original dataframe, fill in only specified columns that have NaN with ''.
final_cohort_file = final_cohort_file.fillna(value = fill_values_pr)

final_cohort_file

,key_nrd,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd_x,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum_x,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year_x,zipinc_qrtl,dxver,prver,hosp_bedsize,h_contrl,hosp_urcat4,hosp_ur_teach,nrd_stratum_y,n_disc_u,n_hosp_u,s_disc_u,s_hosp_u,total_disc,year_y,aprdrg,aprdrg_risk_mortality,aprdrg_severity,hosp_nrd_y,next_nrdday,days_to_readmission,readmission_diagnosis,readmission_los,preventable_admission,pqi_1,pqi_3,pqi_5,pqi_7,pqi_8,pqi_11,pqi_12,pqi_14,pqi_15,pqi_16,pqi_5_15_exclusion,pqi_7_medical_exclusions,pqi_7_procedural_exclusion,pqi_7_8_procedural_exclusion,pqi_11_exclusion,pqi_12_exclusion,pqi_immunocompromise_medical_exclusion,pqi_immunocompromise_procedural_exclusion,pqi_16_exclusion_procedure,pqi_16_amputation_procedure,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,cc_MI,cc_CHF,cc_PVD,cc_CVD,cc_dementia,cc_COPD,cc_rheum,cc_peptic_ulcer,cc_mild_liver_disease,cc_diabetes_wo_complications,cc_diabetes_w_complications,cc_plegia,cc_renal,cc_malignancy,cc_severe_liver,cc_metastasis,cc_AIDS,ibd_diagnosis,ibd_index_hospitalization
0,2.017114e+11,88.0,0.0,0.0,1.805447,1.0,5.0,2.0,470.0,34.0,470.0,M1712,I10,L309,K5190,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.0,0.0,0.0,42214.0,1.0,8.0,8.0,4.0,1.0,13976.0,447.0,e0007w8,1.0,3.0,0SRD0J9,,,,,,,,,,,,,,,,,,,,,,,,,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,43848.0,2017.0,2.0,10.0,10.0,3.0,1.0,2.0,1.0,447,636191,24,332444,15,21737,2017,302.0,1.0,2.0,42214.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
1,2.017057e+11,49.0,1.0,0.0,1.492757,1.0,6.0,2.0,386.0,34.0,386.0,K50812,G40909,J449,F17210,F329,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,0.0,1.0,41827.0,3.0,6.0,6.0,5.0,0.0,19296.0,452.0,e0009vi,2.0,6.0,,,,,,,,,,,,,,,,,,,,,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,11700.0,2017.0,2.0,10.0,10.0,2.0,2.0,4.0,2.0,452,18409,9,11959,7,2872,2017,245.0,1.0,1.0,41827.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.000000,0.0,1.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
2,2.017025e+11,51.0,0.0,0.0,1.631724,7.0,1.0,1.0,872.0,34.0,872.0,A419,I10,Z5321,A09,G8929,Z9884,E669,D62,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,1.0,1.0,44540.0,1.0,18.0,18.0,8.0,0.0,13632.0,453.0,e000fgg,1.0,5.0,,,,,,,,,,,,,,,,,,,,,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,3770.0,2017.0,1.0,10.0,10.0,3.0,1.0,2.0,1.0,453,604585,17,345400,12,28083,2017

## Diagnosis Codes

In [26]:
# Creating columns using the function above.

# -- SERIOUS INFECTIONS -- #

# Intestinal abscess
icd10_to_column(final_cohort_file, diagnoses, 'intestinal_abscess', 'K630')

# Meningitis
icd10_to_column(final_cohort_file, diagnoses, 'meningitis', 'G00|G01|G02|G03|A87')

# Encephalitis
icd10_to_column(final_cohort_file, diagnoses, 'encephalitis', 'G040|G042|G043|G0481|G0490')

# Endocarditis
icd10_to_column(final_cohort_file, diagnoses, 'endocarditis', 'I33|I38|I39')

# Pneumonia
icd10_to_column(final_cohort_file, diagnoses, 'pneumonia', 'J11|J12|J13|J14|J15|J16|J17')

# Pyelonephritis
icd10_to_column(final_cohort_file, diagnoses, 'pyelonephritis', 'N10')

# Septic arthritis, Osteomyelitis
icd10_to_column(final_cohort_file, diagnoses, 'septic_arthritis_osteomyelitis', 'M00|M01|M86')

# Septicemia or bacteremia
icd10_to_column(final_cohort_file, diagnoses, 'septicemia_or_bacteremia', 'A41|R7881')

# Clostridium difficile
icd10_to_column(final_cohort_file, diagnoses, 'clostridium_difficile', 'A047')

# -- OPPORTUNISTIC INFECTIONS -- #

# Pulmonary tuberculosis
icd10_to_column(final_cohort_file, diagnoses, 'pulmonary_tuberculosis', 'A15')

# Atypical mycobacteria
icd10_to_column(final_cohort_file, diagnoses, 'atypical_mycobacteria', 'A31')

# Cryptococcosis, Aspergillosis, Histoplasmosis
icd10_to_column(final_cohort_file, diagnoses, 'cryptococcosis_aspergillosis_histoplasmosis', 'B45|B44|B39')

# Listeriosis
icd10_to_column(final_cohort_file, diagnoses, 'listeriosis', 'A32')

# Leishmaniasis
icd10_to_column(final_cohort_file, diagnoses, 'leishmaniasis', 'B55')

# Pneumocystis jiroveci pneumonia
icd10_to_column(final_cohort_file, diagnoses, 'pneumocystis_jiroveci_pneumonia', 'B59')

# -- CARDIAC -- #

# Acute myocardial infarction
icd10_to_column(final_cohort_file, diagnoses, 'acute_myocardial_infarction', 'I21')

# Heart failure
icd10_to_column(final_cohort_file, diagnoses, 'heart_failure', 'I50')

# Unstable angina
icd10_to_column(final_cohort_file, diagnoses, 'unstable_angina', 'I200')

# Arrhythmia
icd10_to_column(final_cohort_file, diagnoses, 'arrhythmia', 'I44|I45|I47|I48|I49')

# Cardiac arrest
icd10_to_column(final_cohort_file, diagnoses, 'cardiac_arrest', 'I46')

# -- VENOUS THROMBOEMBOLISM -- #

# VTE diagnosis (PE or DVT)
icd10_to_column(final_cohort_file, diagnoses, 'vte_diagnosis_pe_or_dvt', 'I824|I260|I269')

# History of VTE
icd10_to_column(final_cohort_file, diagnoses, 'history_of_vte', 'Z8671')

# -- STEROID USE -- #

# Systemic steroid use
icd10_to_column(final_cohort_file, diagnoses, 'systemic_steroid_use', 'Z7952')

# -- CHRONIC PAIN -- #

# Chronic pain
icd10_to_column(final_cohort_file, diagnoses, 'chronic_pain', 'G894')

# -- OPIOID USE -- #

# Opioid overuse
icd10_to_column(final_cohort_file, diagnoses, 'opioid_overuse', 'F111|T400')

# -- PSYCHIATRIC BURDEN -- #

# Depression
icd10_to_column(final_cohort_file, diagnoses, 'depression', 'F32|F33|F34')

# Anxiety disorder
icd10_to_column(final_cohort_file, diagnoses, 'anxiety_disorder', 'F40|F41')

# Bipolar disorder
icd10_to_column(final_cohort_file, diagnoses, 'bipolar_disorder', 'F31')

# Schizophrenia
icd10_to_column(final_cohort_file, diagnoses, 'schizophrenia', 'F20|F25')

# -- SOCIAL DETERMINANTS OF HEALTH -- #

# Problems related to education and literacy
icd10_to_column(final_cohort_file, diagnoses, 'problems_related_to_education_and_literacy', 'Z55')

# Problems related to employment and unemployment
icd10_to_column(final_cohort_file, diagnoses, 'problems_related_to_employment_and_unemployment', 'Z56')

# Occupational exposure to risk factors
icd10_to_column(final_cohort_file, diagnoses, 'occupational_exposure_to_risk_factors', 'Z57')

# Problems related to housing and economic circumstances
icd10_to_column(final_cohort_file, diagnoses, 'problems_related_to_housing_and_economic_circumstances', 'Z59')

# Problems related to social environment
icd10_to_column(final_cohort_file, diagnoses, 'problems_related_to_social_environment', 'Z60')

# Problems related to upbringing
icd10_to_column(final_cohort_file, diagnoses, 'problems_related_to_upbringing', 'Z62')

# Other problems related to primary support group, including family circumstances
icd10_to_column(final_cohort_file, diagnoses, 'other_problems_related_to_primary_support_group_including_family_circumstances', 'Z63')

# Problems related to certain psychosocial circumstances
icd10_to_column(final_cohort_file, diagnoses, 'problems_related_to_certain_psychosocial_circumstances', 'Z64')

# Problems related to other psychosocial circumstances
icd10_to_column(final_cohort_file, diagnoses, 'problems_related_to_other_psychosocial_circumstances', 'Z65')

Column for intestinal_abscess created.
Column for meningitis created.
Column for encephalitis created.
Column for endocarditis created.
Column for pneumonia created.
Column for pyelonephritis created.
Column for septic_arthritis_osteomyelitis created.
Column for septicemia_or_bacteremia created.
Column for clostridium_difficile created.
Column for pulmonary_tuberculosis created.
Column for atypical_mycobacteria created.
Column for cryptococcosis_aspergillosis_histoplasmosis created.
Column for listeriosis created.
Column for leishmaniasis created.
Column for pneumocystis_jiroveci_pneumonia created.
Column for acute_myocardial_infarction created.
Column for heart_failure created.
Column for unstable_angina created.
Column for arrhythmia created.
Column for cardiac_arrest created.
Column for vte_diagnosis_pe_or_dvt created.
Column for history_of_vte created.
Column for systemic_steroid_use created.
Column for chronic_pain created.
Column for opioid_overuse created.
Column for depression 

In [27]:
final_cohort_file

,key_nrd,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd_x,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum_x,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year_x,zipinc_qrtl,dxver,prver,hosp_bedsize,h_contrl,hosp_urcat4,hosp_ur_teach,nrd_stratum_y,n_disc_u,n_hosp_u,s_disc_u,s_hosp_u,total_disc,year_y,aprdrg,aprdrg_risk_mortality,aprdrg_severity,hosp_nrd_y,next_nrdday,days_to_readmission,readmission_diagnosis,readmission_los,preventable_admission,pqi_1,pqi_3,pqi_5,pqi_7,pqi_8,pqi_11,pqi_12,pqi_14,pqi_15,pqi_16,pqi_5_15_exclusion,pqi_7_medical_exclusions,pqi_7_procedural_exclusion,pqi_7_8_procedural_exclusion,pqi_11_exclusion,pqi_12_exclusion,pqi_immunocompromise_medical_exclusion,pqi_immunocompromise_procedural_exclusion,pqi_16_exclusion_procedure,pqi_16_amputation_procedure,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,cc_MI,cc_CHF,cc_PVD,cc_CVD,cc_dementia,cc_COPD,cc_rheum,cc_peptic_ulcer,cc_mild_liver_disease,cc_diabetes_wo_complications,cc_diabetes_w_complications,cc_plegia,cc_renal,cc_malignancy,cc_severe_liver,cc_metastasis,cc_AIDS,ibd_diagnosis,ibd_index_hospitalization,intestinal_abscess,meningitis,encephalitis,endocarditis,pneumonia,pyelonephritis,septic_arthritis_osteomyelitis,septicemia_or_bacteremia,clostridium_difficile,pulmonary_tuberculosis,atypical_mycobacteria,cryptococcosis_aspergillosis_histoplasmosis,listeriosis,leishmaniasis,pneumocystis_jiroveci_pneumonia,acute_myocardial_infarction,heart_failure,unstable_angina,arrhythmia,cardiac_arrest,vte_diagnosis_pe_or_dvt,history_of_vte,systemic_steroid_use,chronic_pain,opioid_overuse,depression,anxiety_disorder,bipolar_disorder,schizophrenia,problems_related_to_education_and_literacy,problems_related_to_employment_and_unemployment,occupational_exposure_to_risk_factors,problems_related_to_housing_and_economic_circumstances,problems_related_to_social_environment,problems_related_to_upbringing,other_problems_related_to_primary_support_group_including_family_circumstances,problems_related_to_certain_psychosocial_circumstances,problems_related_to_other_psychosocial_circumstances
0,2.017114e+11,88.0,0.0,0.0,1.805447,1.0,5.0,2.0,470.0,34.0,470.0,M1712,I10,L309,K5190,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.0,0.0,0.0,42214.0,1.0,8.0,8.0,4.0,1.0,13976.0,447.0,e0007w8,1.0,3.0,0SRD0J9,,,,,,,,,,,,,,,,,,,,,,,,,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,43848.0,2017.0,2.0,10.0,10.0,3.0,1.0,2.0,1.0,447,636191,24,332444,15,21737,2017,302.0,1.0,2.0,42214.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2.017057e+11,49.0,1.0,0.0,1.492757,1.0,6.0,2.0,386.0,34.0,386.0,K50812,G40909,J449,F17210,F329,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<

## Procedure Codes

In [28]:
# -- SURGERY -- #

# Intestinal surgery
icd10_to_column(final_cohort_file, procedures, 'intestinal_surgery', '0DB80ZZ|0DB83ZZ|0DB84ZZ|0DB87ZZ|0DB88ZZ|0DT90ZZ|0DT94ZZ|0DT97ZZ|0DT98ZZ|0DTA0ZZ|0DTA4ZZ|0DTA7ZZ|0DTA8ZZ|0DTB0ZZ|0DTB4ZZ|0DTB7ZZ|0DTB8ZZ|0DT80ZZ|0DT84ZZ|0DT87ZZ|0DT88ZZ|0DBE0ZZ|0DBE3ZZ|0DBE7ZZ|0DBE8ZZ|0DTH0ZZ|0DTH7ZZ|0DTH8ZZ|0DTF0ZZ|0DTF7ZZ|0DTF8ZZ|0DTK0ZZ|0DTL0ZZ|0DTL7ZZ|0DTL8ZZ|0DTG0ZZ|0DTG7ZZ|0DTG8ZZ|0DTN0ZZ|0DTN7ZZ|0DTN8ZZ|0DBE0ZZ|0DBE3ZZ|0DBE7ZZ|0DBE8ZZ|0DTE4ZZ|0DTE0ZZ|0DTE7ZZ|0DTE8ZZ|0DTP0ZZ|0DTP4ZZ|0DTP7ZZ|0DTP8ZZ|0DTP4ZZ|0DTP0ZZ|0DTP0ZZ|0DTP4ZZ|0D1N0Z4|0D1N4Z4|0DBP0ZZ|0DBP4ZZ|0DQ80ZZ|0DQ83ZZ|0DQ84ZZ|0DQ87ZZ|0DQ88ZZ|0DQA0ZZ|0DQA3ZZ|0DQA4ZZ|0DQA7ZZ|0DQA8ZZ|0DQE0ZZ|0DQE3ZZ|0DQE4ZZ|0DQE7ZZ|0DQE8ZZ|0DQN0ZZ|0DQN3ZZ|0DQN4ZZ|0DQN7ZZ|0DQN8ZZ|0DQP0ZZ|0DQP3ZZ|0DQP4ZZ|0DQP7ZZ|0DQP8ZZ|0DQQ0ZZ|0DQQ3ZZ|0DQQ4ZZ|0DQQ7ZZ|0DQQ8ZZ|0DQQXZZ|0H89XZZ|0DBQ0ZZ|0DBQ3ZZ|0DBQ4ZZ|0TQB0ZZ|0TQB3ZZ|0TQB4ZZ|0TQB7ZZ|0TQB8ZZ|0UQG0ZZ|0UQG3ZZ|0UQG4ZZ|0UQG7ZZ|0UQG8ZZ')

# -- PROCEDURES -- #

# Flexible sigmoidoscopy/colonoscopy
icd10_to_column(final_cohort_file, procedures, 'flexible_sigmoidoscopy_colonoscopy', '0DJD8ZZ|0D9E8ZX|0D9H8ZX|0D9N8ZX|0DBE8ZX|0DBH8ZX|0DBN8ZX|0DBE8ZZ')

# Blood Transfusion
icd10_to_column(final_cohort_file, procedures, 'blood_transfusion', '30233H1|30243H1|30253H1|30263H1|30233H0|30233N0|30233W0|30243H0|30243N0|30243W0|30253H0|30253N0|30253W0|30263H0|30263N0|30263W0|30233N1|30233P1|30243N1|30243P1|30253N1|30253P1|30263N1|30263P1|30233R1|30243R1|30253R1|30263R1')

# Parenteral or enteral nutrition
icd10_to_column(final_cohort_file, procedures, 'parenteral_or_enteral_nutrition', '0DH67UZ|0DH68UZ|3E0G36Z|3E0336Z|3E0436Z|3E0536Z|3E0636Z')

Column for intestinal_surgery created.
Column for flexible_sigmoidoscopy_colonoscopy created.
Column for blood_transfusion created.
Column for parenteral_or_enteral_nutrition created.


## These will be brought in on the original merge using Charleson Comorbidity IDC-10 Codes

### Comorbidities (these use Elixhauser Comorbidity ICD-10 codes: https://cran.r-project.org/web/packages/comorbidity/vignettes/comorbidityscores.html#icd-10-codes-1)

In [29]:
# # cm_aids
# icd10_to_column(final_cohort_file, diagnoses, 'cm_aids', 'B20|B21|B22|B24')

# # cm_alcohol
# icd10_to_column(final_cohort_file, diagnoses, 'cm_alcohol', 'F10|E52|G621|I426|K292|K700|K703|K709|T51|Z502|Z714|Z721')

# # cm_anemdef
# icd10_to_column(final_cohort_file, diagnoses, 'cm_anemdef', 'D508|D509|D51|D52|D53')

# # cm_arth
# icd10_to_column(final_cohort_file, diagnoses, 'cm_arth', 'L940|L941|L943|M05|M06|M08|M120|M123|M30|M310|M311|M312|M313|M32|M33|M34|M35|M45|M461|M468|M469')

# # cm_bldloss
# icd10_to_column(final_cohort_file, diagnoses, 'cm_bldloss', 'D500')

# # cm_chf
# icd10_to_column(final_cohort_file, diagnoses, 'cm_chf', 'I099|I110|I130|I132|I255|I420|I425|I426|I427|I428|I429|I43|I50|P290')

# # cm_chrnlung
# icd10_to_column(final_cohort_file, diagnoses, 'cm_chrnlung', 'I278|I279|J40|J41|J42|J43|J44|J45|J46|J47|J60|J61|J62|J63|J64|J65|J66|J67|J684|J701|J703')

# # cm_coag
# icd10_to_column(final_cohort_file, diagnoses, 'cm_coag', 'D65|D66|D67|D68|D691|D693|D694|D695|D696')

# # cm_depress
# icd10_to_column(final_cohort_file, diagnoses, 'cm_depress', 'F204|F313|F314|F315|F32|F33|F341|F412|F432')

# # cm_dm
# icd10_to_column(final_cohort_file, diagnoses, 'cm_dm', 'E100|E101|E109|E110|E111|E119|E120|E121|E129|E130|E131|E139|E140|E141|E149')

# # cm_dmcx
# icd10_to_column(final_cohort_file, diagnoses, 'cm_dmcx', 'E102|E103|E104|E105|E106|E107|E108|E112|E113|E114|E115|E116|E117|E118|E122|E123|E124|E125|E126|E127|E128|E132|E133|E134|E135|E136|E137|E138|E142|E143|E144|E145|E146|E147|E148')

# # cm_drug
# icd10_to_column(final_cohort_file, diagnoses, 'cm_drug', 'F11|F12|F13|F14|F15|F16|F18|F19|Z715|Z722')

# # cm_htn_c
# icd10_to_column(final_cohort_file, diagnoses, 'cm_htn_c', 'I10|I11|I12|I13|I15')

# # cm_hypothy
# icd10_to_column(final_cohort_file, diagnoses, 'cm_hypothy', 'E00|E01|E02|E03|E890')

# # cm_liver
# icd10_to_column(final_cohort_file, diagnoses, 'cm_liver', 'B18|I85|I864|I982|K70|K711|K713|K714|K715|K717|K72|K73|K74|K760|K762|K763|K764|K765|K766|K767|K768|K769|Z944')

# # cm_lymph
# icd10_to_column(final_cohort_file, diagnoses, 'cm_lymph', 'C81|C82|C83|C84|C85|C88|C96|C900|C902')

# # cm_lytes
# icd10_to_column(final_cohort_file, diagnoses, 'cm_lytes', 'E222|E86|E87')

# # cm_mets
# icd10_to_column(final_cohort_file, diagnoses, 'cm_mets', 'C77|C78|C79|C80')

# # cm_neuro
# icd10_to_column(final_cohort_file, diagnoses, 'cm_neuro', 'G10|G11|G12|G13|G20|G21|G22|G254|G255|G312|G318|G319|G32|G35|G36|G37|G40|G41|G931|G934|R470|R56')

# # cm_obese
# icd10_to_column(final_cohort_file, diagnoses, 'cm_obese', 'E66')

# # cm_para
# icd10_to_column(final_cohort_file, diagnoses, 'cm_para', 'G041|G114|G801|G802|G81|G82|G830|G831|G832|G833|G834|G839')

# # cm_perivasc
# icd10_to_column(final_cohort_file, diagnoses, 'cm_perivasc', 'I70|I71|I731|I738|I739|I771|I790|I792|K551|K558|K559|Z958|Z959')

# # cm_psych
# icd10_to_column(final_cohort_file, diagnoses, 'cm_psych', 'F20|F22|F23|F24|F25|F28|F29|F302|F312|F315')

# # cm_pulmcirc
# icd10_to_column(final_cohort_file, diagnoses, 'cm_pulmcirc', 'I26|I27|I280|I288|I289')

# # cm_renlfail
# icd10_to_column(final_cohort_file, diagnoses, 'cm_renlfail', 'I120|I131|N18|N19|N250|Z490|Z491|Z492|Z940|Z992')

# # cm_tumor
# icd10_to_column(final_cohort_file, diagnoses, 'cm_tumor', 'C00|C01|C02|C03|C04|C05|C06|C07|C08|C09|C10|C11|C12|C13|C14|C15|C16|C17|C18|C19|C20|C21|C22|C23|C24|C25|C26|C30|C31|C32|C33|C34|C37|C38|C39|C40|C41|C43|C45|C46|C47|C48|C49|C50|C51|C52|C53|C54|C55|C56|C57|C58|C60|C61|C62|C63|C64|C65|C66|C67|C68|C69|C70|C71|C72|C73|C74|C75|C76|C97')

# # cm_ulcer
# icd10_to_column(final_cohort_file, diagnoses, 'cm_ulcer', 'K257|K259|K267|K269|K277|K279|K287|K289')

# # cm_valve
# icd10_to_column(final_cohort_file, diagnoses, 'cm_valve', 'A520|I05|I06|I07|I08|I091|I098|I34|I35|I36|I37|I38|I39|Q230|Q231|Q232|Q233|Z952|Z953|Z954')

# # cm_wghtloss
# icd10_to_column(final_cohort_file, diagnoses, 'cm_wghtloss', 'E40|E41|E42|E43|E44|E45|E46|R634|R64')

## These will be brought in on the original merge
### AHRQ PQI

In [30]:
# # -- PQI 1 -- #
# # Diabetes short term complications
# icd10_to_column(final_cohort_file, diagnoses, 'diabetes_short_term_complications', 'E1010|E1011|E10641|E1065|E1100|E1101|E1110|E1111|E11641')

# # -- PQI 3 -- #
# # Diabetes long term complications
# icd10_to_column(final_cohort_file, diagnoses, 'diabetes_long_term_complications', 'E10349|E10351|E10359|E1036|E1039|E1040|E1041|E1042|E1043|E1044|E1049|E1051|E1052|E1059|E10610|E10618|E10620|E10621|E10622|E10628|E10630|E10638|E1069|E108|E1121|E1122|E1129|E11311|E11319|E11321|E11329|E11331|E11339|E11341|E11349|E11351|E11359|E1136|E1139|E1140|E1141|E1142|E1143|E1144|E1149|E1151|E1152|E1159|E11610|E11618|E11620|E11621|E11622|E11628|E11630|E11638|E1169|E118|E1321|E1322|E1329|E13311|E13319|E13321|E13329|E13331|E13339|E13341|E13349|E13351|E13359|E1336|E1339|E1340|E1341|E1342|E1343|E1344|E1349|E1351|E1352|E1359|E13610|E13618|E13620|E13621|E13622|E13628|E13630|E13638|E13649|E1365|E1369|E138')

# # -- PQI 5 -- #
# # COPD (chronic obstructive pulmonary disease)
# icd10_to_column(final_cohort_file, diagnoses, 'copd', 'J410|J411|J418|J42|J430|J431|J432|J438|J439|J440|J441|J449|J470|J471|J479')

# # Adult asthma
# icd10_to_column(final_cohort_file, diagnoses, 'adult_asthma', 'J452|J4521|J4522|J4530|J4531|J4532|J4540|J4541|J4542|J4550|J4551|J4552|J45901|J45902|J45909|J45990|J45991|J45998')

# # For adult_asthma, exclude any listed ICD-10 for cystic fibrosis with the diagnosis codes below by overriding any value of 1 to 0 in the 'adult_asthma' column.
# for column in diagnoses:
#     final_cohort_file.loc[final_cohort_file[column].str.contains('E840|E8411|E8419|E848|E849|E8483|E84841|E849|J8483|J84841|J84842|J84843|J84848|P270|P271|P278|P279|Q254|Q311|Q312|Q313|Q315|Q318|Q319|Q320|Q321|Q322|Q323|Q324|Q33|Q34|Q39|Q693'), 'adult_asthma'] = 0

# # # Cystic fibrosis
# icd10_to_column(final_cohort_file, diagnoses, 'cystic_fibrosis', 'E840|E8411|E8419|E848|E849|E8483|E84841|E849|J8483|J84841|J84842|J84843|J84848|P270|P271|P278|P279|Q254|Q311|Q312|Q313|Q315|Q318|Q319|Q320|Q321|Q322|Q323|Q324|Q33|Q34|Q39|Q693')

# # -- PQI 7 -- #
# # Hypertension
# icd10_to_column(final_cohort_file, diagnoses, 'hypertension', 'I10|I119|I129|I1310')

# # For hypertension, exclude any with ICD-10 diagnosis of stage I-IV kidney disease IF accompanied by a ICD-10 procedural code for dialysis access.
# # If diagnoses contains kidney disease and procedures contains dialysis access, overide the 1 in the 'hypertension' column with 0.
# for dx_column in diagnoses:
#     for pr_column in procedures:
#         final_cohort_file.loc[(final_cohort_file[dx_column].str.contains('I129|I1310')) & (final_cohort_file[pr_column].str.contains('031509D|031509D|03170ZV|03170ZV|031509F|031509F|03170ZW|03170ZW|031509V|031509V|031809D|031809D|031509W|031509W|031809F|031809F|03150AD|03150AD|031809V|031809V|03150AF|03150AF|031809W|031809W|03150AV|03150AV|03180AD|03180AD|03150AW|03150AW|03180AF|03180AF|03150JD|03150JD|03180AV|03180AV|03150JF|03150JF|03180AW|03180AW|03150JV|03150JV|03180JD|03180JD|03150JW|03150JW|03180JF|03180JF|03150KD|03150KD|03180JV|03180JV|03150KF|03150KF|03180JW|03180JW|03150KV|03150KV|03180KD|03180KD|03150KW|03150KW|03180KF|03180KF|03150ZD|03150ZD|03180KV|03180KV|03150ZF|03150ZF|03180KW|03180KW|03150ZV|03150ZV|03180ZD|03180ZD|03150ZW|03150ZW|03180ZF|03180ZF|031609D|031609D|03180ZV|03180ZV|031609F|031609F|03180ZW|03180ZW|031609V|031609V|031909F|031909F|031609W|031609W|03190AF|03190AF|03160AD|03160AD|03190JF|03190JF|03160AF|03160AF|03190KF|03190KF|03160AV|03160AV|03190ZF|03190ZF|03160AW|03160AW|03193ZF|03193ZF|03160JD|03160JD|031A09F|031A09F|03160JF|03160JF|031A0AF|031A0AF|03160JV|03160JV|031A0JF|031A0JF|03160JW|03160JW|031A0KF|031A0KF|03160KD|03160KD|031A0ZF|031A0ZF|03160KF|03160KF|031A3ZF|031A3ZF|03160KV|03160KV|031B09F|031B09F|03160KW|03160KW|031B0AF|031B0AF|03160ZD|03160ZD|031B0JF|031B0JF|03160ZF|03160ZF|031B0KF|031B0KF|03160ZV|03160ZV|031B0ZF|031B0ZF|03160ZW|03160ZW|031B3ZF|031B3ZF|031709D|031709D|031C09F|031C09F|031709F|031709F|031C0AF|031C0AF|031709V|031709V|031C0JF|031C0JF|031709W|031709W|031C0KF|031C0KF|03170AD|03170AD|031C0ZF|031C0ZF|03170AF|03170AF|031C3ZF|031C3ZF|03170AV|03170AV|041K09S|041K09S|03170AW|03170AW|041K0AS|041K0AS|03170JD|03170JD|041K0JS|041K0JS|03170JF|03170JF|041K0KS|041K0KS|03170JV|03170JV|041K0ZS|041K0ZS|03170JW|03170JW|041K3JS|041K3JS|03170KD|03170KD|041L09S|041L09S|03170KF|03170KF|041L0AS|041L0AS|03170KV|03170KV|041L0JS|041L0JS|03170KW|03170KW|041L0KS|041L0KS|03170ZD|03170ZD|041L0ZS|041L0ZS|03170ZF|03170ZF|041L3JS|041L3JS')), 'hypertension'] = 0

# # Special exclusion for PQI 7, any with ICD-10 procedural code of cardiac access.
# for column in procedures:
#     final_cohort_file.loc[final_cohort_file[column].str.contains('02C03ZZ|0210088|02C04Z6|0210089|02C04ZZ|021008C|02C10Z6|021008F|02C10ZZ|021008W|02C13Z6|0210093|02C13ZZ|0210098|02C14Z6|0210099|02C14ZZ|021009C|02C20Z6|021009F|02C20ZZ|021009W|02C23Z6|02100A3|02C23ZZ|02100A8|02C24Z6|02100A9|02C24ZZ|02100AC|02C30Z6|02100AF|02C30ZZ|02100AW|02C33Z6|02100J3|02C33ZZ|02100J8|02C34Z6|02100J9|02C34ZZ|02100JC|02C40ZZ|02100JF|02C43ZZ|02100JW|02C44ZZ|02100K3|02C50ZZ|02100K8|02C53ZZ|02100K9|02C54ZZ|02100KC|02C60ZZ|02100KF|02C64ZZ|02100KW|02C70ZZ|02100Z3|02C74ZZ|02100Z8|02C80ZZ|02100Z9|02C84ZZ|02100ZC|02C90ZZ|02100ZF|02C94ZZ|0210344|02CD0ZZ|02103D4|02CD3ZZ|0210444|02CD4ZZ|0210483|02CF0ZZ|0210488|02CF3ZZ|0210489|02CF4ZZ|021048C|02CG0ZZ|021048F|02CG3ZZ|021048W|02CG4ZZ|0210493|02CH0ZZ|0210498|02CH3ZZ|0210499|02CH4ZZ|021049C|02CJ0ZZ|021049F|02CJ3ZZ|021049W|02CJ4ZZ|02104A3|02CK0ZZ|02104A8|02CK4ZZ|02104A9|02CL0ZZ|02104AC|02CL4ZZ|02104AF|02CM0ZZ|02104AW|02CM3ZZ|02104D4|02CM4ZZ|02104J3|02CN0ZZ|02104J8|02H00DZ|02104J9|02H00YZ|02104JC|02H03DZ|02104JF|02H03YZ|02104JW|02H04DZ|02104K3|02H04YZ|02104K8|02H10DZ|02104K9|02H10YZ|02104KC|02H13DZ|02104KF|02H13YZ|02104KW|02H14DZ|02104Z3|02H14YZ|02104Z8|02H20DZ|02104Z9|02H20YZ|02104ZC|02H23DZ|02104ZF|02H23YZ|0211083|02H24DZ|0211088|02H24YZ|0211089|02H30DZ|021108C|02H30YZ|021108F|02H33DZ|021108W|02H33YZ|0211093|02H34DZ|0211098|02H34YZ|0211099|02H400Z|021109C|02H402Z|021109F|02H403Z|021109W|02H40DZ|02110A3|02H40JZ|02110A8|02H40KZ|02110A9|02H40MZ|02110AC|02H40NZ|02110AF|02H430Z|02110AW|02H432Z|02110J3|02H433Z|02110J8|02H43DZ|02110J9|02H43JZ|02110JC|02H43KZ|02110JF|02H43MZ|02110JW|02H43NZ|02110K3|02H440Z|02110K8|02H442Z|02110K9|02H443Z|02110KC|02H44DZ|02110KF|02H44JZ|02110KW|02H44KZ|02110Z3|02H44MZ|02110Z8|02H44NZ|02110Z9|02H600Z|02110ZC|02H60JZ|02110ZF|02H60KZ|0211344|02H60MZ|02113D4|02H60NZ|0211444|02H630Z|0211483|02H63JZ|0211488|02H63KZ|0211489|02H63MZ|021148C|02H640Z|021148F|02H64JZ|021148W|02H64KZ|0211493|02H64MZ|0211498|02H64NZ|0211499|02H700Z|021149C|02H70JZ|021149F|02H70KZ|021149W|02H70MZ|02114A3|02H70NZ|02114A8|02H730Z|02114A9|02H73JZ|02114AC|02H73KZ|02114AF|02H73MZ|02114AW|02H740Z|02114D4|02H74JZ|02114J3|02H74KZ|02114J8|02H74MZ|02114J9|02H74NZ|02114JC|02HA0QZ|02114JF|02HA0RJ|02114JW|02HA0RS|02114K3|02HA0RZ|02114K8|02HA3QZ|02114K9|02HA3RJ|02114KC|02HA3RS|02114KF|02HA3RZ|02114KW|02HA4QZ|02114Z3|02HA4RJ|02114Z8|02HA4RS|02114Z9|02HA4RZ|02114ZC|02HK00Z|02114ZF|02HK02Z|0212083|02HK0JZ|0212088|02HK0KZ|0212089|02HK0MZ|021208C|02HK0NZ|021208F|02HK30Z|021208W|02HK32Z|0212093|02HK3JZ|0212098|02HK3KZ|0212099|02HK3MZ|021209C|02HK40Z|021209F|02HK42Z|021209W|02HK4JZ|02120A3|02HK4KZ|02120A8|02HK4MZ|02120A9|02HK4NZ|02120AC|02HL00Z|02120AF|02HL0JZ|02120AW|02HL0KZ|02120J3|02HL0MZ|02120J8|02HL0NZ|02120J9|02HL30Z|02120JC|02HL3JZ|02120JF|02HL3KZ|02120JW|02HL3MZ|02120K3|02HL40Z|02120K8|02HL4JZ|02120K9|02HL4KZ|02120KC|02HL4MZ|02120KF|02HL4NZ|02120KW|02HN0JZ|02120Z3|02HN0KZ|02120Z8|02HN0MZ|02120Z9|02HN3JZ|02120ZC|02HN3KZ|02120ZF|02HN3MZ|0212344|02HN4JZ|02123D4|02HN4KZ|0212444|02HN4MZ|0212483|02HS00Z|0212488|02HS30Z|0212489|02HS40Z|021248C|02HT00Z|021248F|02HT30Z|021248W|02HT40Z|0212493|02HV00Z|0212498|02HV30Z|0212499|02HV40Z|021249C|02HX02Z|021249F|02HX0DZ|021249W|02HX42Z|02124A3|02HX4DZ|02124A8|02L70CK|02124A9|02L70DK|02124AC|02L70ZK|02124AF|02L73CK|02124AW|02L73DK|02124D4|02L73ZK|02124J3|02L74CK|02124J8|02L74DK|02124J9|02L74ZK|02124JC|02LQ0CZ|02124JF|02LQ0DZ|02124JW|02LQ0ZZ|02124K3|02LQ3CZ|02124K8|02LQ3DZ|02124K9|02LQ3ZZ|02124KC|02LQ4CZ|02124KF|02LQ4DZ|02124KW|02LQ4ZZ|02124Z3|02LR0CZ|02124Z8|02LR0DZ|02124Z9|02LR0ZT|02124ZC|02LR0ZZ|02124ZF|02LR3CZ|0213083|02LR3DZ|0213088|02LR3ZZ|0213089|02LR4CZ|021308C|02LR4DZ|021308F|02LR4ZZ|021308W|02LS0ZZ|0213093|02LT0ZZ|0213098|02N00ZZ|0213099|02N03ZZ|021309C|02N04ZZ|021309F|02N10ZZ|021309W|02N13ZZ|02130A3|02N14ZZ|02130A8|02N20ZZ|02130A9|02N23ZZ|02130AC|02N24ZZ|02130AF|02N30ZZ|02130AW|02N33ZZ|02130J3|02N34ZZ|02130J8|02N50ZZ|02130J9|02N53ZZ|02130JC|02N54ZZ|02130JF|02N90ZZ|02130JW|02N93ZZ|02130K3|02N94ZZ|02130K8|02ND0ZZ|02130K9|02ND3ZZ|02130KC|02ND4ZZ|02130KF|02NF0ZZ|02130KW|02NF3ZZ|02130Z3|02NF4ZZ|02130Z8|02NG0ZZ|02130Z9|02NG3ZZ|02130ZC|02NG4ZZ|02130ZF|02NH0ZZ|0213344|02NH3ZZ|02133D4|02NH4ZZ|0213444|02NJ0ZZ|0213483|02NJ3ZZ|0213488|02NJ4ZZ|0213489|02NK0ZZ|021348C|02NK3ZZ|021348F|02NK4ZZ|021348W|02NL0ZZ|0213493|02NL3ZZ|0213498|02NL4ZZ|0213499|02NM0ZZ|021349C|02NM3ZZ|021349F|02NM4ZZ|021349W|02PA0MZ|02134A3|02PA0NZ|02134A8|02PA0QZ|02134A9|02PA0RS|02134AC|02PA0RZ|02134AF|02PA3MZ|02134AW|02PA3QZ|02134D4|02PA3RS|02134J3|02PA3RZ|02134J8|02PA4MZ|02134J9|02PA4NZ|02134JC|02PA4QZ|02134JF|02PA4RS|02134JW|02PA4RZ|02134K3|02PAXMZ|02134K8|02Q00ZZ|02134K9|02Q03ZZ|02134KC|02Q04ZZ|02134KF|02Q10ZZ|02134KW|02Q13ZZ|02134Z3|02Q14ZZ|02134Z8|02Q20ZZ|02134Z9|02Q23ZZ|02134ZC|02Q24ZZ|02134ZF|02Q30ZZ|021608P|02Q33ZZ|021608Q|02Q34ZZ|021608R|02Q40ZZ|021609P|02Q43ZZ|021609Q|02Q44ZZ|021609R|02Q50ZZ|02160AP|02Q53ZZ|02160AQ|02Q54ZZ|02160AR|02Q70ZZ|02160JP|02Q73ZZ|02160JQ|02Q74ZZ|02160JR|02Q90ZZ|02160KP|02Q93ZZ|02160KQ|02Q94ZZ|02160KR|02QA0ZZ|02160Z7|02QA3ZZ|02160ZP|02QA4ZZ|02160ZQ|02QB0ZZ|02160ZR|02QB3ZZ|02163Z7|02QB4ZZ|021648P|02QC0ZZ|021648Q|02QC3ZZ|021648R|02QC4ZZ|021649P|02QD0ZZ|021649Q|02QD3ZZ|021649R|02QD4ZZ|02164AP|02QF0ZJ|02164AQ|02QF0ZZ|02164AR|02QF3ZJ|02164JP|02QF3ZZ|02164JQ|02QF4ZJ|02164JR|02QF4ZZ|02164KP|02QG0ZE|02164KQ|02QG0ZZ|02164KR|02QG3ZE|02164Z7|02QG3ZZ|02164ZP|02QG4ZE|02164ZQ|02QG4ZZ|02164ZR|02QH0ZZ|021708P|02QH3ZZ|021708Q|02QH4ZZ|021708R|02QJ0ZG|021709P|02QJ0ZZ|021709Q|02QJ3ZG|021709R|02QJ3ZZ|021709S|02QJ4ZG|021709T|02QJ4ZZ|021709U|02QM0ZZ|02170AP|02QM3ZZ|02170AQ|02QM4ZZ|02170AR|02R907Z|02170AS|02R908Z|02170AT|02R90JZ|02170AU|02R90KZ|02170JP|02R947Z|02170JQ|02R948Z|02170JR|02R94JZ|02170JS|02R94KZ|02170JT|02RD07Z|02170JU|02RD08Z|02170KP|02RD0JZ|02170KQ|02RD0KZ|02170KR|02RD47Z|02170KS|02RD48Z|02170KT|02RD4JZ|02170KU|02RD4KZ|02170ZP|02RF07Z|02170ZQ|02RF08Z|02170ZR|02RF0JZ|02170ZS|02RF0KZ|02170ZT|02RF37H|02170ZU|02RF37Z|021748P|02RF38H|021748Q|02RF38Z|021748R|02RF3JH|021749P|02RF3JZ|021749Q|02RF3KH|021749R|02RF3KZ|021749S|02RF47Z|021749T|02RF48Z|021749U|02RF4JZ|02174AP|02RF4KZ|02174AQ|02RG07Z|02174AR|02RG08Z|02174AS|02RG0JZ|02174AT|02RG0KZ|02174AU|02RG37H|02174JP|02RG37Z|02174JQ|02RG38H|02174JR|02RG38Z|02174JS|02RG3JH|02174JT|02RG3JZ|02174JU|02RG3KH|02174KP|02RG3KZ|02174KQ|02RG47Z|02174KR|02RG48Z|02174KS|02RG4JZ|02174KT|02RG4KZ|02174KU|02RH07Z|02174ZP|02RH08Z|02174ZQ|02RH0JZ|02174ZR|02RH0KZ|02174ZS|02RH37H|02174ZT|02RH37Z|02174ZU|02RH38H|021K08P|02RH38Z|021K08Q|02RH3JH|021K08R|02RH3JZ|021K09P|02RH3KH|021K09Q|02RH3KZ|021K09R|02RH47Z|021K0AP|02RH48Z|021K0AQ|02RH4JZ|021K0AR|02RH4KZ|021K0JP|02RJ07Z|021K0JQ|02RJ08Z|021K0JR|02RJ0JZ|021K0KP|02RJ0KZ|021K0KQ|02RJ37H|021K0KR|02RJ37Z|021K0Z5|02RJ38H|021K0Z8|02RJ38Z|021K0Z9|02RJ3JH|021K0ZC|02RJ3JZ|021K0ZF|02RJ3KH|021K0ZP|02RJ3KZ|021K0ZQ|02RJ47Z|021K0ZR|02RJ48Z|021K0ZW|02RJ4JZ|021K48P|02RJ4KZ|021K48Q|02RK07Z|021K48R|02RK0JZ|021K49P|02RK0KZ|021K49Q|02RK47Z|021K49R|02RK4KZ|021K4AP|02RL07Z|021K4AQ|02RL0JZ|021K4AR|02RL0KZ|021K4JP|02RL47Z|021K4JQ|02RL4KZ|021K4JR|02RM07Z|021K4KP|02RM0JZ|021K4KQ|02RM0KZ|021K4KR|02RM47Z|021K4Z5|02RM4JZ|021K4Z8|02RM4KZ|021K4Z9|02RP0JZ|021K4ZC|02RQ07Z|021K4ZF|02RQ0JZ|021K4ZP|02RR07Z|021K4ZQ|02RR0JZ|021K4ZR|02SP0ZZ|021K4ZW|02SW0ZZ|021L08P|02SX0ZZ|021L08Q|02T50ZZ|021L08R|02T53ZZ|021L09P|02T54ZZ|021L09Q|02T80ZZ|021L09R|02T83ZZ|021L0AP|02T84ZZ|021L0AQ|02T90ZZ|021L0AR|02T93ZZ|021L0JP|02T94ZZ|021L0JQ|02TD0ZZ|021L0JR|02TD3ZZ|021L0KP|02TD4ZZ|021L0KQ|02TH0ZZ|021L0KR|02TH3ZZ|021L0Z5|02TH4ZZ|021L0Z8|02TM0ZZ|021L0Z9|02TM3ZZ|021L0ZC|02TM4ZZ|021L0ZF|02TN0ZZ|021L0ZP|02TN3ZZ|021L0ZQ|02TN4ZZ|021L0ZR|02U007Z|021L0ZW|02U008Z|021L48P|02U00JZ|021L48Q|02U00KZ|021L48R|02U037Z|021L49P|02U038Z|021L49Q|02U03JZ|021L49R|02U03KZ|021L4AP|02U047Z|021L4AQ|02U048Z|021L4AR|02U04JZ|021L4JP|02U04KZ|021L4JQ|02U107Z|021L4JR|02U108Z|021L4KP|02U10JZ|021L4KQ|02U10KZ|021L4KR|02U137Z|021L4Z5|02U138Z|021L4Z8|02U13JZ|021L4Z9|02U13KZ|021L4ZC|02U147Z|021L4ZF|02U148Z|021L4ZP|02U14JZ|021L4ZQ|02U14KZ|021L4ZR|02U207Z|021L4ZW|02U208Z|024F07J|02U20JZ|024F08J|02U20KZ|024F0JJ|02U237Z|024F0KJ|02U238Z|024G072|02U23JZ|024G082|02U23KZ|024G0J2|02U247Z|024G0K2|02U248Z|024J072|02U24JZ|024J082|02U24KZ|024J0J2|02U307Z|024J0K2|02U308Z|02540ZZ|02U30JZ|02543ZZ|02U30KZ|02544ZZ|02U337Z|02550ZZ|02U338Z|02553ZZ|02U33JZ|02554ZZ|02U33KZ|02560ZZ|02U347Z|02563ZZ|02U348Z|02564ZZ|02U34JZ|02570ZK|02U34KZ|02570ZZ|02U507Z|02573ZK|02U508Z|02573ZZ|02U50JZ|02574ZK|02U50KZ|02574ZZ|02U537Z|02580ZZ|02U538Z|02583ZZ|02U53JZ|02584ZZ|02U53KZ|02590ZZ|02U547Z|02593ZZ|02U548Z|02594ZZ|02U54JZ|025D0ZZ|02U54KZ|025D3ZZ|02U607Z|025D4ZZ|02U608Z|025F0ZZ|02U60KZ|025F3ZZ|02U707Z|025F4ZZ|02U708Z|025G0ZZ|02U70JZ|025G3ZZ|02U70KZ|025G4ZZ|02U737Z|025H0ZZ|02U738Z|025H3ZZ|02U73KZ|025H4ZZ|02U747Z|025J0ZZ|02U748Z|025J3ZZ|02U74KZ|025J4ZZ|02U907Z|025K0ZZ|02U908Z|025K3ZZ|02U90JZ|025K4ZZ|02U90KZ|025L0ZZ|02U937Z|025L3ZZ|02U938Z|025L4ZZ|02U93JZ|025M0ZZ|02U93KZ|025M3ZZ|02U947Z|025M4ZZ|02U948Z|025N0ZZ|02U94JZ|025N3ZZ|02U94KZ|025N4ZZ|02UA0JZ|0270046|02UA3JZ|027004Z|02UA4JZ|0270056|02UD07Z|027005Z|02UD08Z|0270066|02UD0JZ|027006Z|02UD0KZ|0270076|02UD37Z|027007Z|02UD38Z|02700D6|02UD3JZ|02700DZ|02UD3KZ|02700E6|02UD47Z|02700EZ|02UD48Z|02700F6|02UD4JZ|02700FZ|02UD4KZ|02700G6|02UF07J|02700GZ|02UF07Z|02700T6|02UF08J|02700TZ|02UF08Z|02700Z6|02UF0JJ|02700ZZ|02UF0JZ|0270346|02UF0KJ|027034Z|02UF0KZ|0270356|02UF37J|027035Z|02UF37Z|0270366|02UF38J|027036Z|02UF38Z|0270376|02UF3JJ|027037Z|02UF3JZ|02703D6|02UF3KJ|02703DZ|02UF3KZ|02703E6|02UF47J|02703EZ|02UF47Z|02703F6|02UF48J|02703FZ|02UF48Z|02703G6|02UF4JJ|02703GZ|02UF4JZ|02703T6|02UF4KJ|02703TZ|02UF4KZ|02703Z6|02UG07E|02703ZZ|02UG07Z|0270446|02UG08E|027044Z|02UG08Z|0270456|02UG0JE|027045Z|02UG0JZ|0270466|02UG0KE|027046Z|02UG0KZ|0270476|02UG37E|027047Z|02UG37Z|02704D6|02UG38E|02704DZ|02UG38Z|02704E6|02UG3JE|02704EZ|02UG3JZ|02704F6|02UG3KE|02704FZ|02UG3KZ|02704G6|02UG47E|02704GZ|02UG47Z|02704T6|02UG48E|02704TZ|02UG48Z|02704Z6|02UG4JE|02704ZZ|02UG4JZ|0271046|02UG4KE|027104Z|02UG4KZ|0271056|02UH07Z|027105Z|02UH08Z|0271066|02UH0JZ|027106Z|02UH0KZ|0271076|02UH37Z|027107Z|02UH38Z|02710D6|02UH3JZ|02710DZ|02UH3KZ|02710E6|02UH47Z|02710EZ|02UH48Z|02710F6|02UH4JZ|02710FZ|02UH4KZ|02710G6|02UJ07G|02710GZ|02UJ07Z|02710T6|02UJ08G|02710TZ|02UJ08Z|02710Z6|02UJ0JG|02710ZZ|02UJ0JZ|0271346|02UJ0KG|027134Z|02UJ0KZ|0271356|02UJ37G|027135Z|02UJ37Z|0271366|02UJ38G|027136Z|02UJ38Z|0271376|02UJ3JG|027137Z|02UJ3JZ|02713D6|02UJ3KG|02713DZ|02UJ3KZ|02713E6|02UJ47G|02713EZ|02UJ47Z|02713F6|02UJ48G|02713FZ|02UJ48Z|02713G6|02UJ4JG|02713GZ|02UJ4JZ|02713T6|02UJ4KG|02713TZ|02UJ4KZ|02713Z6|02UK0KZ|02713ZZ|02UK3KZ|0271446|02UK4KZ|027144Z|02UL0KZ|0271456|02UL3KZ|027145Z|02UL4KZ|0271466|02UM07Z|027146Z|02UM0JZ|0271476|02UM0KZ|027147Z|02UM37Z|02714D6|02UM38Z|02714DZ|02UM3JZ|02714E6|02UM3KZ|02714EZ|02UM47Z|02714F6|02UM48Z|02714FZ|02UM4JZ|02714G6|02UM4KZ|02714GZ|02VG0ZZ|02714T6|02VG3ZZ|02714TZ|02VG4ZZ|02714Z6|02VR0ZT|02714ZZ|02W50JZ|0272046|02W54JZ|027204Z|02WA0JZ|0272056|02WA0MZ|027205Z|02WA0NZ|0272066|02WA0QZ|027206Z|02WA0RS|0272076|02WA0RZ|027207Z|02WA3MZ|02720D6|02WA3QZ|02720DZ|02WA3RS|02720E6|02WA3RZ|02720EZ|02WA4MZ|02720F6|02WA4NZ|02720FZ|02WA4QZ|02720G6|02WA4RS|02720GZ|02WA4RZ|02720T6|02WAXRS|02720TZ|02WF07Z|02720Z6|02WF08Z|02720ZZ|02WF0JZ|0272346|02WF0KZ|027234Z|02WF37Z|0272356|02WF38Z|027235Z|02WF3JZ|0272366|02WF3KZ|027236Z|02WF47Z|0272376|02WF48Z|027237Z|02WF4JZ|02723D6|02WF4KZ|02723DZ|02WG07Z|02723E6|02WG08Z|02723EZ|02WG0JZ|02723F6|02WG0KZ|02723FZ|02WG37Z|02723G6|02WG38Z|02723GZ|02WG3JZ|02723T6|02WG3KZ|02723TZ|02WG47Z|02723Z6|02WG48Z|02723ZZ|02WG4JZ|0272446|02WG4KZ|027244Z|02WH07Z|0272456|02WH08Z|027245Z|02WH0JZ|0272466|02WH0KZ|027246Z|02WH37Z|0272476|02WH38Z|027247Z|02WH3JZ|02724D6|02WH3KZ|02724DZ|02WH47Z|02724E6|02WH48Z|02724EZ|02WH4JZ|02724F6|02WH4KZ|02724FZ|02WJ07Z|02724G6|02WJ08Z|02724GZ|02WJ0JZ|02724T6|02WJ0KZ|02724TZ|02WJ37Z|02724Z6|02WJ38Z|02724ZZ|02WJ3JZ|0273046|02WJ3KZ|027304Z|02WJ47Z|0273056|02WJ48Z|027305Z|02WJ4JZ|0273066|02WJ4KZ|027306Z|02WM0JZ|0273076|02WM4JZ|027307Z|02YA0Z0|02730D6|02YA0Z1|02730DZ|02YA0Z2|02730E6|061007P|02730EZ|061007Q|02730F6|061007R|02730FZ|061009P|02730G6|061009Q|02730GZ|061009R|02730T6|06100AP|02730TZ|06100AQ|02730Z6|06100AR|02730ZZ|06100JP|0273346|06100JQ|027334Z|06100JR|0273356|06100KP|027335Z|06100KQ|0273366|06100KR|027336Z|06100ZP|0273376|06100ZQ|027337Z|06100ZR|02733D6|061047P|02733DZ|061047Q|02733E6|061047R|02733EZ|061049P|02733F6|061049Q|02733FZ|061049R|02733G6|06104AP|02733GZ|06104AQ|02733T6|06104AR|02733TZ|06104JP|02733Z6|06104JQ|02733ZZ|06104JR|0273446|06104KP|027344Z|06104KQ|0273456|06104KR|027345Z|06104ZP|0273466|06104ZQ|027346Z|06104ZR|0273476|0JH600Z|027347Z|0JH604Z|02734D6|0JH605Z|02734DZ|0JH606Z|02734E6|0JH607Z|02734EZ|0JH608Z|02734F6|0JH609Z|02734FZ|0JH60AZ|02734G6|0JH60FZ|02734GZ|0JH60PZ|02734T6|0JH630Z|02734TZ|0JH634Z|02734Z6|0JH635Z|02734ZZ|0JH636Z|027F04Z|0JH637Z|027F0DZ|0JH638Z|027F0ZZ|0JH639Z|027F34Z|0JH63AZ|027F3DZ|0JH63FZ|027F3ZZ|0JH63PZ|027F44Z|0JH800Z|027F4DZ|0JH804Z|027F4ZZ|0JH805Z|027G04Z|0JH806Z|027G0DZ|0JH807Z|027G0ZZ|0JH808Z|027G34Z|0JH809Z|027G3DZ|0JH80AZ|027G3ZZ|0JH80PZ|027G44Z|0JH830Z|027G4DZ|0JH834Z|027G4ZZ|0JH835Z|027H04Z|0JH836Z|027H0DZ|0JH837Z|027H0ZZ|0JH838Z|027H34Z|0JH839Z|027H3DZ|0JH83AZ|027H3ZZ|0JH83PZ|027H44Z|0JPT0FZ|027H4DZ|0JPT0PZ|027H4ZZ|0JPT3FZ|027J04Z|0JPT3PZ|027J0DZ|0JWT0FZ|027J0ZZ|0JWT0PZ|027J34Z|0JWT3FZ|027J3DZ|0JWT3PZ|027J3ZZ|0JWTXFZ|027J44Z|0W9D00Z|027J4DZ|0W9D0ZX|027J4ZZ|0W9D0ZZ|027K04Z|0W9D30Z|027K0DZ|0W9D3ZX|027K0ZZ|0W9D3ZZ|027K44Z|0W9D40Z|027K4DZ|0W9D4ZX|027K4ZZ|0W9D4ZZ|027R04T|0WCD0ZZ|027R0DT|0WCD3ZZ|027R0ZT|0WCD4ZZ|027R44T|0WFD0ZZ|027R4DT|0WFD3ZZ|027R4ZT|0WFD4ZZ|02890ZZ|0WFDXZZ|02893ZZ|0WHD03Z|02894ZZ|0WHD0YZ|028D0ZZ|0WHD33Z|028D3ZZ|0WHD3YZ|028D4ZZ|0WHD43Z|02B40ZZ|0WHD4YZ|02B43ZZ|0WPD00Z|02B44ZZ|0WPD01Z|02B50ZZ|0WPD03Z|02B53ZZ|0WPD0YZ|02B54ZZ|0WPD30Z|02B60ZZ|0WPD31Z|02B63ZZ|0WPD33Z|02B64ZZ|0WPD3YZ|02B70ZK|0WPD40Z|02B70ZZ|0WPD41Z|02B73ZK|0WPD43Z|02B73ZZ|0WPD4YZ|02B74ZK|0WWD00Z|02B74ZZ|0WWD01Z|02B80ZZ|0WWD03Z|02B83ZZ|0WWD0YZ|02B84ZZ|0WWD30Z|02B90ZZ|0WWD31Z|02B93ZZ|0WWD33Z|02B94ZZ|0WWD3YZ|02BD0ZZ|0WWD40Z|02BD3ZZ|0WWD41Z|02BD4ZZ|0WWD43Z|02BF0ZZ|0WWD4YZ|02BF3ZZ|3E07016|02BF4ZZ|3E07017|02BG0ZZ|3E070GC|02BG3ZZ|3E070KZ|02BG4ZZ|3E070PZ|02BH0ZZ|3E07316|02BH3ZZ|3E07317|02BH4ZZ|3E073GC|02BJ0ZZ|3E073KZ|02BJ3ZZ|3E073PZ|02BJ4ZZ|3E074GC|02BK0ZZ|5A02110|02BK3ZZ|5A02115|02BK4ZZ|5A02116|02BL0ZZ|5A0211D|02BL3ZZ|5A02210|02BL4ZZ|5A02215|02BM0ZZ|5A02216|02BM3ZZ|5A0221D|02BM4ZZ|X2C0361|02BN0ZZ|X2C1361|02BN3ZZ|X2C2361|02BN4ZZ|X2C3361|02C00Z6|X2RF032|02C00ZZ|X2RF332|02C03Z6|X2RF432'), 'hypertension'] = 0

# # -- PQI 8 -- #
# # CHF (congestive heart failure)
# icd10_to_column(final_cohort_file, diagnoses, 'chf', 'I0981|I501|I5020|I5021|I5022|I5023|I5030|I5031|I5032|I5033|I5040|I5041|I5042|I5043|I509')

# # Special exclusion for PQI 8, any with ICD-10 procedural code of cardiac access.
# for column in procedures:
#     final_cohort_file.loc[final_cohort_file[column].str.contains('02C03ZZ|0210088|02C04Z6|0210089|02C04ZZ|021008C|02C10Z6|021008F|02C10ZZ|021008W|02C13Z6|0210093|02C13ZZ|0210098|02C14Z6|0210099|02C14ZZ|021009C|02C20Z6|021009F|02C20ZZ|021009W|02C23Z6|02100A3|02C23ZZ|02100A8|02C24Z6|02100A9|02C24ZZ|02100AC|02C30Z6|02100AF|02C30ZZ|02100AW|02C33Z6|02100J3|02C33ZZ|02100J8|02C34Z6|02100J9|02C34ZZ|02100JC|02C40ZZ|02100JF|02C43ZZ|02100JW|02C44ZZ|02100K3|02C50ZZ|02100K8|02C53ZZ|02100K9|02C54ZZ|02100KC|02C60ZZ|02100KF|02C64ZZ|02100KW|02C70ZZ|02100Z3|02C74ZZ|02100Z8|02C80ZZ|02100Z9|02C84ZZ|02100ZC|02C90ZZ|02100ZF|02C94ZZ|0210344|02CD0ZZ|02103D4|02CD3ZZ|0210444|02CD4ZZ|0210483|02CF0ZZ|0210488|02CF3ZZ|0210489|02CF4ZZ|021048C|02CG0ZZ|021048F|02CG3ZZ|021048W|02CG4ZZ|0210493|02CH0ZZ|0210498|02CH3ZZ|0210499|02CH4ZZ|021049C|02CJ0ZZ|021049F|02CJ3ZZ|021049W|02CJ4ZZ|02104A3|02CK0ZZ|02104A8|02CK4ZZ|02104A9|02CL0ZZ|02104AC|02CL4ZZ|02104AF|02CM0ZZ|02104AW|02CM3ZZ|02104D4|02CM4ZZ|02104J3|02CN0ZZ|02104J8|02H00DZ|02104J9|02H00YZ|02104JC|02H03DZ|02104JF|02H03YZ|02104JW|02H04DZ|02104K3|02H04YZ|02104K8|02H10DZ|02104K9|02H10YZ|02104KC|02H13DZ|02104KF|02H13YZ|02104KW|02H14DZ|02104Z3|02H14YZ|02104Z8|02H20DZ|02104Z9|02H20YZ|02104ZC|02H23DZ|02104ZF|02H23YZ|0211083|02H24DZ|0211088|02H24YZ|0211089|02H30DZ|021108C|02H30YZ|021108F|02H33DZ|021108W|02H33YZ|0211093|02H34DZ|0211098|02H34YZ|0211099|02H400Z|021109C|02H402Z|021109F|02H403Z|021109W|02H40DZ|02110A3|02H40JZ|02110A8|02H40KZ|02110A9|02H40MZ|02110AC|02H40NZ|02110AF|02H430Z|02110AW|02H432Z|02110J3|02H433Z|02110J8|02H43DZ|02110J9|02H43JZ|02110JC|02H43KZ|02110JF|02H43MZ|02110JW|02H43NZ|02110K3|02H440Z|02110K8|02H442Z|02110K9|02H443Z|02110KC|02H44DZ|02110KF|02H44JZ|02110KW|02H44KZ|02110Z3|02H44MZ|02110Z8|02H44NZ|02110Z9|02H600Z|02110ZC|02H60JZ|02110ZF|02H60KZ|0211344|02H60MZ|02113D4|02H60NZ|0211444|02H630Z|0211483|02H63JZ|0211488|02H63KZ|0211489|02H63MZ|021148C|02H640Z|021148F|02H64JZ|021148W|02H64KZ|0211493|02H64MZ|0211498|02H64NZ|0211499|02H700Z|021149C|02H70JZ|021149F|02H70KZ|021149W|02H70MZ|02114A3|02H70NZ|02114A8|02H730Z|02114A9|02H73JZ|02114AC|02H73KZ|02114AF|02H73MZ|02114AW|02H740Z|02114D4|02H74JZ|02114J3|02H74KZ|02114J8|02H74MZ|02114J9|02H74NZ|02114JC|02HA0QZ|02114JF|02HA0RJ|02114JW|02HA0RS|02114K3|02HA0RZ|02114K8|02HA3QZ|02114K9|02HA3RJ|02114KC|02HA3RS|02114KF|02HA3RZ|02114KW|02HA4QZ|02114Z3|02HA4RJ|02114Z8|02HA4RS|02114Z9|02HA4RZ|02114ZC|02HK00Z|02114ZF|02HK02Z|0212083|02HK0JZ|0212088|02HK0KZ|0212089|02HK0MZ|021208C|02HK0NZ|021208F|02HK30Z|021208W|02HK32Z|0212093|02HK3JZ|0212098|02HK3KZ|0212099|02HK3MZ|021209C|02HK40Z|021209F|02HK42Z|021209W|02HK4JZ|02120A3|02HK4KZ|02120A8|02HK4MZ|02120A9|02HK4NZ|02120AC|02HL00Z|02120AF|02HL0JZ|02120AW|02HL0KZ|02120J3|02HL0MZ|02120J8|02HL0NZ|02120J9|02HL30Z|02120JC|02HL3JZ|02120JF|02HL3KZ|02120JW|02HL3MZ|02120K3|02HL40Z|02120K8|02HL4JZ|02120K9|02HL4KZ|02120KC|02HL4MZ|02120KF|02HL4NZ|02120KW|02HN0JZ|02120Z3|02HN0KZ|02120Z8|02HN0MZ|02120Z9|02HN3JZ|02120ZC|02HN3KZ|02120ZF|02HN3MZ|0212344|02HN4JZ|02123D4|02HN4KZ|0212444|02HN4MZ|0212483|02HS00Z|0212488|02HS30Z|0212489|02HS40Z|021248C|02HT00Z|021248F|02HT30Z|021248W|02HT40Z|0212493|02HV00Z|0212498|02HV30Z|0212499|02HV40Z|021249C|02HX02Z|021249F|02HX0DZ|021249W|02HX42Z|02124A3|02HX4DZ|02124A8|02L70CK|02124A9|02L70DK|02124AC|02L70ZK|02124AF|02L73CK|02124AW|02L73DK|02124D4|02L73ZK|02124J3|02L74CK|02124J8|02L74DK|02124J9|02L74ZK|02124JC|02LQ0CZ|02124JF|02LQ0DZ|02124JW|02LQ0ZZ|02124K3|02LQ3CZ|02124K8|02LQ3DZ|02124K9|02LQ3ZZ|02124KC|02LQ4CZ|02124KF|02LQ4DZ|02124KW|02LQ4ZZ|02124Z3|02LR0CZ|02124Z8|02LR0DZ|02124Z9|02LR0ZT|02124ZC|02LR0ZZ|02124ZF|02LR3CZ|0213083|02LR3DZ|0213088|02LR3ZZ|0213089|02LR4CZ|021308C|02LR4DZ|021308F|02LR4ZZ|021308W|02LS0ZZ|0213093|02LT0ZZ|0213098|02N00ZZ|0213099|02N03ZZ|021309C|02N04ZZ|021309F|02N10ZZ|021309W|02N13ZZ|02130A3|02N14ZZ|02130A8|02N20ZZ|02130A9|02N23ZZ|02130AC|02N24ZZ|02130AF|02N30ZZ|02130AW|02N33ZZ|02130J3|02N34ZZ|02130J8|02N50ZZ|02130J9|02N53ZZ|02130JC|02N54ZZ|02130JF|02N90ZZ|02130JW|02N93ZZ|02130K3|02N94ZZ|02130K8|02ND0ZZ|02130K9|02ND3ZZ|02130KC|02ND4ZZ|02130KF|02NF0ZZ|02130KW|02NF3ZZ|02130Z3|02NF4ZZ|02130Z8|02NG0ZZ|02130Z9|02NG3ZZ|02130ZC|02NG4ZZ|02130ZF|02NH0ZZ|0213344|02NH3ZZ|02133D4|02NH4ZZ|0213444|02NJ0ZZ|0213483|02NJ3ZZ|0213488|02NJ4ZZ|0213489|02NK0ZZ|021348C|02NK3ZZ|021348F|02NK4ZZ|021348W|02NL0ZZ|0213493|02NL3ZZ|0213498|02NL4ZZ|0213499|02NM0ZZ|021349C|02NM3ZZ|021349F|02NM4ZZ|021349W|02PA0MZ|02134A3|02PA0NZ|02134A8|02PA0QZ|02134A9|02PA0RS|02134AC|02PA0RZ|02134AF|02PA3MZ|02134AW|02PA3QZ|02134D4|02PA3RS|02134J3|02PA3RZ|02134J8|02PA4MZ|02134J9|02PA4NZ|02134JC|02PA4QZ|02134JF|02PA4RS|02134JW|02PA4RZ|02134K3|02PAXMZ|02134K8|02Q00ZZ|02134K9|02Q03ZZ|02134KC|02Q04ZZ|02134KF|02Q10ZZ|02134KW|02Q13ZZ|02134Z3|02Q14ZZ|02134Z8|02Q20ZZ|02134Z9|02Q23ZZ|02134ZC|02Q24ZZ|02134ZF|02Q30ZZ|021608P|02Q33ZZ|021608Q|02Q34ZZ|021608R|02Q40ZZ|021609P|02Q43ZZ|021609Q|02Q44ZZ|021609R|02Q50ZZ|02160AP|02Q53ZZ|02160AQ|02Q54ZZ|02160AR|02Q70ZZ|02160JP|02Q73ZZ|02160JQ|02Q74ZZ|02160JR|02Q90ZZ|02160KP|02Q93ZZ|02160KQ|02Q94ZZ|02160KR|02QA0ZZ|02160Z7|02QA3ZZ|02160ZP|02QA4ZZ|02160ZQ|02QB0ZZ|02160ZR|02QB3ZZ|02163Z7|02QB4ZZ|021648P|02QC0ZZ|021648Q|02QC3ZZ|021648R|02QC4ZZ|021649P|02QD0ZZ|021649Q|02QD3ZZ|021649R|02QD4ZZ|02164AP|02QF0ZJ|02164AQ|02QF0ZZ|02164AR|02QF3ZJ|02164JP|02QF3ZZ|02164JQ|02QF4ZJ|02164JR|02QF4ZZ|02164KP|02QG0ZE|02164KQ|02QG0ZZ|02164KR|02QG3ZE|02164Z7|02QG3ZZ|02164ZP|02QG4ZE|02164ZQ|02QG4ZZ|02164ZR|02QH0ZZ|021708P|02QH3ZZ|021708Q|02QH4ZZ|021708R|02QJ0ZG|021709P|02QJ0ZZ|021709Q|02QJ3ZG|021709R|02QJ3ZZ|021709S|02QJ4ZG|021709T|02QJ4ZZ|021709U|02QM0ZZ|02170AP|02QM3ZZ|02170AQ|02QM4ZZ|02170AR|02R907Z|02170AS|02R908Z|02170AT|02R90JZ|02170AU|02R90KZ|02170JP|02R947Z|02170JQ|02R948Z|02170JR|02R94JZ|02170JS|02R94KZ|02170JT|02RD07Z|02170JU|02RD08Z|02170KP|02RD0JZ|02170KQ|02RD0KZ|02170KR|02RD47Z|02170KS|02RD48Z|02170KT|02RD4JZ|02170KU|02RD4KZ|02170ZP|02RF07Z|02170ZQ|02RF08Z|02170ZR|02RF0JZ|02170ZS|02RF0KZ|02170ZT|02RF37H|02170ZU|02RF37Z|021748P|02RF38H|021748Q|02RF38Z|021748R|02RF3JH|021749P|02RF3JZ|021749Q|02RF3KH|021749R|02RF3KZ|021749S|02RF47Z|021749T|02RF48Z|021749U|02RF4JZ|02174AP|02RF4KZ|02174AQ|02RG07Z|02174AR|02RG08Z|02174AS|02RG0JZ|02174AT|02RG0KZ|02174AU|02RG37H|02174JP|02RG37Z|02174JQ|02RG38H|02174JR|02RG38Z|02174JS|02RG3JH|02174JT|02RG3JZ|02174JU|02RG3KH|02174KP|02RG3KZ|02174KQ|02RG47Z|02174KR|02RG48Z|02174KS|02RG4JZ|02174KT|02RG4KZ|02174KU|02RH07Z|02174ZP|02RH08Z|02174ZQ|02RH0JZ|02174ZR|02RH0KZ|02174ZS|02RH37H|02174ZT|02RH37Z|02174ZU|02RH38H|021K08P|02RH38Z|021K08Q|02RH3JH|021K08R|02RH3JZ|021K09P|02RH3KH|021K09Q|02RH3KZ|021K09R|02RH47Z|021K0AP|02RH48Z|021K0AQ|02RH4JZ|021K0AR|02RH4KZ|021K0JP|02RJ07Z|021K0JQ|02RJ08Z|021K0JR|02RJ0JZ|021K0KP|02RJ0KZ|021K0KQ|02RJ37H|021K0KR|02RJ37Z|021K0Z5|02RJ38H|021K0Z8|02RJ38Z|021K0Z9|02RJ3JH|021K0ZC|02RJ3JZ|021K0ZF|02RJ3KH|021K0ZP|02RJ3KZ|021K0ZQ|02RJ47Z|021K0ZR|02RJ48Z|021K0ZW|02RJ4JZ|021K48P|02RJ4KZ|021K48Q|02RK07Z|021K48R|02RK0JZ|021K49P|02RK0KZ|021K49Q|02RK47Z|021K49R|02RK4KZ|021K4AP|02RL07Z|021K4AQ|02RL0JZ|021K4AR|02RL0KZ|021K4JP|02RL47Z|021K4JQ|02RL4KZ|021K4JR|02RM07Z|021K4KP|02RM0JZ|021K4KQ|02RM0KZ|021K4KR|02RM47Z|021K4Z5|02RM4JZ|021K4Z8|02RM4KZ|021K4Z9|02RP0JZ|021K4ZC|02RQ07Z|021K4ZF|02RQ0JZ|021K4ZP|02RR07Z|021K4ZQ|02RR0JZ|021K4ZR|02SP0ZZ|021K4ZW|02SW0ZZ|021L08P|02SX0ZZ|021L08Q|02T50ZZ|021L08R|02T53ZZ|021L09P|02T54ZZ|021L09Q|02T80ZZ|021L09R|02T83ZZ|021L0AP|02T84ZZ|021L0AQ|02T90ZZ|021L0AR|02T93ZZ|021L0JP|02T94ZZ|021L0JQ|02TD0ZZ|021L0JR|02TD3ZZ|021L0KP|02TD4ZZ|021L0KQ|02TH0ZZ|021L0KR|02TH3ZZ|021L0Z5|02TH4ZZ|021L0Z8|02TM0ZZ|021L0Z9|02TM3ZZ|021L0ZC|02TM4ZZ|021L0ZF|02TN0ZZ|021L0ZP|02TN3ZZ|021L0ZQ|02TN4ZZ|021L0ZR|02U007Z|021L0ZW|02U008Z|021L48P|02U00JZ|021L48Q|02U00KZ|021L48R|02U037Z|021L49P|02U038Z|021L49Q|02U03JZ|021L49R|02U03KZ|021L4AP|02U047Z|021L4AQ|02U048Z|021L4AR|02U04JZ|021L4JP|02U04KZ|021L4JQ|02U107Z|021L4JR|02U108Z|021L4KP|02U10JZ|021L4KQ|02U10KZ|021L4KR|02U137Z|021L4Z5|02U138Z|021L4Z8|02U13JZ|021L4Z9|02U13KZ|021L4ZC|02U147Z|021L4ZF|02U148Z|021L4ZP|02U14JZ|021L4ZQ|02U14KZ|021L4ZR|02U207Z|021L4ZW|02U208Z|024F07J|02U20JZ|024F08J|02U20KZ|024F0JJ|02U237Z|024F0KJ|02U238Z|024G072|02U23JZ|024G082|02U23KZ|024G0J2|02U247Z|024G0K2|02U248Z|024J072|02U24JZ|024J082|02U24KZ|024J0J2|02U307Z|024J0K2|02U308Z|02540ZZ|02U30JZ|02543ZZ|02U30KZ|02544ZZ|02U337Z|02550ZZ|02U338Z|02553ZZ|02U33JZ|02554ZZ|02U33KZ|02560ZZ|02U347Z|02563ZZ|02U348Z|02564ZZ|02U34JZ|02570ZK|02U34KZ|02570ZZ|02U507Z|02573ZK|02U508Z|02573ZZ|02U50JZ|02574ZK|02U50KZ|02574ZZ|02U537Z|02580ZZ|02U538Z|02583ZZ|02U53JZ|02584ZZ|02U53KZ|02590ZZ|02U547Z|02593ZZ|02U548Z|02594ZZ|02U54JZ|025D0ZZ|02U54KZ|025D3ZZ|02U607Z|025D4ZZ|02U608Z|025F0ZZ|02U60KZ|025F3ZZ|02U707Z|025F4ZZ|02U708Z|025G0ZZ|02U70JZ|025G3ZZ|02U70KZ|025G4ZZ|02U737Z|025H0ZZ|02U738Z|025H3ZZ|02U73KZ|025H4ZZ|02U747Z|025J0ZZ|02U748Z|025J3ZZ|02U74KZ|025J4ZZ|02U907Z|025K0ZZ|02U908Z|025K3ZZ|02U90JZ|025K4ZZ|02U90KZ|025L0ZZ|02U937Z|025L3ZZ|02U938Z|025L4ZZ|02U93JZ|025M0ZZ|02U93KZ|025M3ZZ|02U947Z|025M4ZZ|02U948Z|025N0ZZ|02U94JZ|025N3ZZ|02U94KZ|025N4ZZ|02UA0JZ|0270046|02UA3JZ|027004Z|02UA4JZ|0270056|02UD07Z|027005Z|02UD08Z|0270066|02UD0JZ|027006Z|02UD0KZ|0270076|02UD37Z|027007Z|02UD38Z|02700D6|02UD3JZ|02700DZ|02UD3KZ|02700E6|02UD47Z|02700EZ|02UD48Z|02700F6|02UD4JZ|02700FZ|02UD4KZ|02700G6|02UF07J|02700GZ|02UF07Z|02700T6|02UF08J|02700TZ|02UF08Z|02700Z6|02UF0JJ|02700ZZ|02UF0JZ|0270346|02UF0KJ|027034Z|02UF0KZ|0270356|02UF37J|027035Z|02UF37Z|0270366|02UF38J|027036Z|02UF38Z|0270376|02UF3JJ|027037Z|02UF3JZ|02703D6|02UF3KJ|02703DZ|02UF3KZ|02703E6|02UF47J|02703EZ|02UF47Z|02703F6|02UF48J|02703FZ|02UF48Z|02703G6|02UF4JJ|02703GZ|02UF4JZ|02703T6|02UF4KJ|02703TZ|02UF4KZ|02703Z6|02UG07E|02703ZZ|02UG07Z|0270446|02UG08E|027044Z|02UG08Z|0270456|02UG0JE|027045Z|02UG0JZ|0270466|02UG0KE|027046Z|02UG0KZ|0270476|02UG37E|027047Z|02UG37Z|02704D6|02UG38E|02704DZ|02UG38Z|02704E6|02UG3JE|02704EZ|02UG3JZ|02704F6|02UG3KE|02704FZ|02UG3KZ|02704G6|02UG47E|02704GZ|02UG47Z|02704T6|02UG48E|02704TZ|02UG48Z|02704Z6|02UG4JE|02704ZZ|02UG4JZ|0271046|02UG4KE|027104Z|02UG4KZ|0271056|02UH07Z|027105Z|02UH08Z|0271066|02UH0JZ|027106Z|02UH0KZ|0271076|02UH37Z|027107Z|02UH38Z|02710D6|02UH3JZ|02710DZ|02UH3KZ|02710E6|02UH47Z|02710EZ|02UH48Z|02710F6|02UH4JZ|02710FZ|02UH4KZ|02710G6|02UJ07G|02710GZ|02UJ07Z|02710T6|02UJ08G|02710TZ|02UJ08Z|02710Z6|02UJ0JG|02710ZZ|02UJ0JZ|0271346|02UJ0KG|027134Z|02UJ0KZ|0271356|02UJ37G|027135Z|02UJ37Z|0271366|02UJ38G|027136Z|02UJ38Z|0271376|02UJ3JG|027137Z|02UJ3JZ|02713D6|02UJ3KG|02713DZ|02UJ3KZ|02713E6|02UJ47G|02713EZ|02UJ47Z|02713F6|02UJ48G|02713FZ|02UJ48Z|02713G6|02UJ4JG|02713GZ|02UJ4JZ|02713T6|02UJ4KG|02713TZ|02UJ4KZ|02713Z6|02UK0KZ|02713ZZ|02UK3KZ|0271446|02UK4KZ|027144Z|02UL0KZ|0271456|02UL3KZ|027145Z|02UL4KZ|0271466|02UM07Z|027146Z|02UM0JZ|0271476|02UM0KZ|027147Z|02UM37Z|02714D6|02UM38Z|02714DZ|02UM3JZ|02714E6|02UM3KZ|02714EZ|02UM47Z|02714F6|02UM48Z|02714FZ|02UM4JZ|02714G6|02UM4KZ|02714GZ|02VG0ZZ|02714T6|02VG3ZZ|02714TZ|02VG4ZZ|02714Z6|02VR0ZT|02714ZZ|02W50JZ|0272046|02W54JZ|027204Z|02WA0JZ|0272056|02WA0MZ|027205Z|02WA0NZ|0272066|02WA0QZ|027206Z|02WA0RS|0272076|02WA0RZ|027207Z|02WA3MZ|02720D6|02WA3QZ|02720DZ|02WA3RS|02720E6|02WA3RZ|02720EZ|02WA4MZ|02720F6|02WA4NZ|02720FZ|02WA4QZ|02720G6|02WA4RS|02720GZ|02WA4RZ|02720T6|02WAXRS|02720TZ|02WF07Z|02720Z6|02WF08Z|02720ZZ|02WF0JZ|0272346|02WF0KZ|027234Z|02WF37Z|0272356|02WF38Z|027235Z|02WF3JZ|0272366|02WF3KZ|027236Z|02WF47Z|0272376|02WF48Z|027237Z|02WF4JZ|02723D6|02WF4KZ|02723DZ|02WG07Z|02723E6|02WG08Z|02723EZ|02WG0JZ|02723F6|02WG0KZ|02723FZ|02WG37Z|02723G6|02WG38Z|02723GZ|02WG3JZ|02723T6|02WG3KZ|02723TZ|02WG47Z|02723Z6|02WG48Z|02723ZZ|02WG4JZ|0272446|02WG4KZ|027244Z|02WH07Z|0272456|02WH08Z|027245Z|02WH0JZ|0272466|02WH0KZ|027246Z|02WH37Z|0272476|02WH38Z|027247Z|02WH3JZ|02724D6|02WH3KZ|02724DZ|02WH47Z|02724E6|02WH48Z|02724EZ|02WH4JZ|02724F6|02WH4KZ|02724FZ|02WJ07Z|02724G6|02WJ08Z|02724GZ|02WJ0JZ|02724T6|02WJ0KZ|02724TZ|02WJ37Z|02724Z6|02WJ38Z|02724ZZ|02WJ3JZ|0273046|02WJ3KZ|027304Z|02WJ47Z|0273056|02WJ48Z|027305Z|02WJ4JZ|0273066|02WJ4KZ|027306Z|02WM0JZ|0273076|02WM4JZ|027307Z|02YA0Z0|02730D6|02YA0Z1|02730DZ|02YA0Z2|02730E6|061007P|02730EZ|061007Q|02730F6|061007R|02730FZ|061009P|02730G6|061009Q|02730GZ|061009R|02730T6|06100AP|02730TZ|06100AQ|02730Z6|06100AR|02730ZZ|06100JP|0273346|06100JQ|027334Z|06100JR|0273356|06100KP|027335Z|06100KQ|0273366|06100KR|027336Z|06100ZP|0273376|06100ZQ|027337Z|06100ZR|02733D6|061047P|02733DZ|061047Q|02733E6|061047R|02733EZ|061049P|02733F6|061049Q|02733FZ|061049R|02733G6|06104AP|02733GZ|06104AQ|02733T6|06104AR|02733TZ|06104JP|02733Z6|06104JQ|02733ZZ|06104JR|0273446|06104KP|027344Z|06104KQ|0273456|06104KR|027345Z|06104ZP|0273466|06104ZQ|027346Z|06104ZR|0273476|0JH600Z|027347Z|0JH604Z|02734D6|0JH605Z|02734DZ|0JH606Z|02734E6|0JH607Z|02734EZ|0JH608Z|02734F6|0JH609Z|02734FZ|0JH60AZ|02734G6|0JH60FZ|02734GZ|0JH60PZ|02734T6|0JH630Z|02734TZ|0JH634Z|02734Z6|0JH635Z|02734ZZ|0JH636Z|027F04Z|0JH637Z|027F0DZ|0JH638Z|027F0ZZ|0JH639Z|027F34Z|0JH63AZ|027F3DZ|0JH63FZ|027F3ZZ|0JH63PZ|027F44Z|0JH800Z|027F4DZ|0JH804Z|027F4ZZ|0JH805Z|027G04Z|0JH806Z|027G0DZ|0JH807Z|027G0ZZ|0JH808Z|027G34Z|0JH809Z|027G3DZ|0JH80AZ|027G3ZZ|0JH80PZ|027G44Z|0JH830Z|027G4DZ|0JH834Z|027G4ZZ|0JH835Z|027H04Z|0JH836Z|027H0DZ|0JH837Z|027H0ZZ|0JH838Z|027H34Z|0JH839Z|027H3DZ|0JH83AZ|027H3ZZ|0JH83PZ|027H44Z|0JPT0FZ|027H4DZ|0JPT0PZ|027H4ZZ|0JPT3FZ|027J04Z|0JPT3PZ|027J0DZ|0JWT0FZ|027J0ZZ|0JWT0PZ|027J34Z|0JWT3FZ|027J3DZ|0JWT3PZ|027J3ZZ|0JWTXFZ|027J44Z|0W9D00Z|027J4DZ|0W9D0ZX|027J4ZZ|0W9D0ZZ|027K04Z|0W9D30Z|027K0DZ|0W9D3ZX|027K0ZZ|0W9D3ZZ|027K44Z|0W9D40Z|027K4DZ|0W9D4ZX|027K4ZZ|0W9D4ZZ|027R04T|0WCD0ZZ|027R0DT|0WCD3ZZ|027R0ZT|0WCD4ZZ|027R44T|0WFD0ZZ|027R4DT|0WFD3ZZ|027R4ZT|0WFD4ZZ|02890ZZ|0WFDXZZ|02893ZZ|0WHD03Z|02894ZZ|0WHD0YZ|028D0ZZ|0WHD33Z|028D3ZZ|0WHD3YZ|028D4ZZ|0WHD43Z|02B40ZZ|0WHD4YZ|02B43ZZ|0WPD00Z|02B44ZZ|0WPD01Z|02B50ZZ|0WPD03Z|02B53ZZ|0WPD0YZ|02B54ZZ|0WPD30Z|02B60ZZ|0WPD31Z|02B63ZZ|0WPD33Z|02B64ZZ|0WPD3YZ|02B70ZK|0WPD40Z|02B70ZZ|0WPD41Z|02B73ZK|0WPD43Z|02B73ZZ|0WPD4YZ|02B74ZK|0WWD00Z|02B74ZZ|0WWD01Z|02B80ZZ|0WWD03Z|02B83ZZ|0WWD0YZ|02B84ZZ|0WWD30Z|02B90ZZ|0WWD31Z|02B93ZZ|0WWD33Z|02B94ZZ|0WWD3YZ|02BD0ZZ|0WWD40Z|02BD3ZZ|0WWD41Z|02BD4ZZ|0WWD43Z|02BF0ZZ|0WWD4YZ|02BF3ZZ|3E07016|02BF4ZZ|3E07017|02BG0ZZ|3E070GC|02BG3ZZ|3E070KZ|02BG4ZZ|3E070PZ|02BH0ZZ|3E07316|02BH3ZZ|3E07317|02BH4ZZ|3E073GC|02BJ0ZZ|3E073KZ|02BJ3ZZ|3E073PZ|02BJ4ZZ|3E074GC|02BK0ZZ|5A02110|02BK3ZZ|5A02115|02BK4ZZ|5A02116|02BL0ZZ|5A0211D|02BL3ZZ|5A02210|02BL4ZZ|5A02215|02BM0ZZ|5A02216|02BM3ZZ|5A0221D|02BM4ZZ|X2C0361|02BN0ZZ|X2C1361|02BN3ZZ|X2C2361|02BN4ZZ|X2C3361|02C00Z6|X2RF032|02C00ZZ|X2RF332|02C03Z6|X2RF432'), 'chf'] = 0

# # -- PQI 11 -- #
# # Bacterial Pneumonia
# icd10_to_column(final_cohort_file, diagnoses, 'bacterial_pneumonia', 'J13|J14|J15211|J15212|J153|J154|J157|J159|J160|J168|J180|J181|J188|J189')

# # For bacterial_pneumonia, exclude any listed ICD-10 for sickle cell with the diagnosis codes below by overriding any value of 1 to 0 in the 'bacterial_pneumonia' column.
# for column in diagnoses:
#     final_cohort_file.loc[final_cohort_file[column].str.contains('D57'), 'bacterial_pneumonia'] = 0

# # # Sickle cell pneumonia
# icd10_to_column(final_cohort_file, diagnoses, 'sickle_cell_pneumonia', 'D57')

# # Special exclusions for PQI 11, any with ICD-10 diagnosis code or ICD-10 procedural code that indicate immunocomprimised state.
# for column in diagnoses:
#     final_cohort_file.loc[final_cohort_file[column].str.contains('B20|E41|B59|E42|C802|E43|C888|I120|C9440|I1311|C9441|I132|C9442|K912|C946|N185|D4622|N186|D4701|T8600|D4702|T8601|D4709|T8602|D471|T8603|D479|T8609|D47Z1|T8610|D47Z2|T8611|D47Z9|T8612|D6109|T8613|D61810|T8619|v2020|D61811|T8620|D61818|T8621|D700|T8622|D701|T8623|D702|T86290|D704|T86298|D708|T8630|D709|T8631|D71|T8632|D720|T8633|D72810|T8639|D72818|T8640|D72819|T8641|D7381|T8642|D7581|T8643|D761|T8649|D762|T865|D763|T86810|D800|T86811|D801|T86812|D802|T86818|D803|T86819|D804|T86830|D805|T86831|D806|T86832|D807|T86838|D808|T86839|v2020|D809|T86850|D810|T86851|D811|T86852|D812|T86858|D8130|T86859|D8131|T86890|D8132|T86891|D8139|T86892|D814|T86898|D816|T86899|D817|T8690|D8189|T8691|D819|T8692|D820|T8693|D821|T8699|D822|Z4821|D823|Z4822|D824|Z4823|D828|Z4824|D829|Z48280|v2020|D830|Z48288|D831|Z48290|D832|Z48298|D838|Z4901|D839|Z4902|D840|Z4931|D841|Z4932|D848|Z940|D849|Z941|D893|Z942|D89810|Z943|D89811|Z944|D89812|Z9481|D89813|Z9482|D8982|Z9483|D8989|Z9484|D899|Z9489|E40|Z992'), 'bacterial_pneumonia'] = 0

# for column in procedures:
#     final_cohort_file.loc[final_cohort_file[column].str.contains('30233Y4|02YA0Z2|30240AZ|0BYC0Z0|30240G0|0BYC0Z2|30240G1|0BYD0Z0|30240G2|0BYD0Z2|30240G3|0BYF0Z0|30240G4|0BYF0Z2|30240U2|0BYG0Z0|30240U3|0BYG0Z2|30240U4|0BYH0Z0|30240X0|0BYH0Z2|30240X1|0BYJ0Z0|30240X2|0BYJ0Z2|30240X3|0BYK0Z0|30240X4|0BYK0Z2|30240Y0|0BYL0Z0|30240Y1|0BYL0Z2|30240Y2|0BYM0Z0|30240Y3|0BYM0Z2|30240Y4|0DY50Z0|30243AZ|0DY50Z2|30243G0|0DY60Z0|30243G1|0DY60Z2|30243G2|0DY80Z0|30243G3|0DY80Z2|30243G4|0DYE0Z0|30243U2|0DYE0Z2|30243U3|0FY00Z0|30243U4|0FY00Z2|30243X0|0FYG0Z0|30243X1|0FYG0Z2|30243X2|0TY00Z0|30243X3|0TY00Z2|30243X4|0TY10Z0|30243Y0|0TY10Z2|30243Y1|0WY20Z0|30243Y2|0XYJ0Z0|30243Y3|0XYK0Z0|30243Y4|30230AZ|3E03005|30230G0|3E0300M|30230G1|3E0300P|30230G2|3E030U1|30230G3|3E030WL|30230G4|3E03305|30230U2|3E0330M|30230U3|3E0330P|30230U4|3E033U1|30230X0|3E033WL|30230X1|3E04005|30230X2|3E0400M|30230X3|3E0400P|30230X4|3E040WL|30230Y0|3E04305|30230Y1|3E0430M|30230Y2|3E0430P|30230Y3|3E043WL|30230Y4|3E0A305|30233AZ|3E0A30M|30233G0|3E0D705|30233G1|3E0G705|30233G2|3E0J3U1|30233G3|3E0J7U1|30233G4|3E0J8U1|30233U2|XW03351|30233U3|XW033B3|30233U4|XW033Q5|30233X0|XW033S5|30233X1|XW04351|30233X2|XW043B3|30233X3|XW043Q5|30233X4|XW043S5|30233Y0|XW0DXJ5|30233Y1|XW0DXL5|30233Y2|XW0DXR5|30233Y3|XW0DXT5|XW0DXV5'), 'bacterial_pneumonia'] = 0


# # -- PQI 12 -- #
# # UTI (urinary tract infection)
# icd10_to_column(final_cohort_file, diagnoses, 'uti', 'N10|N119|N12|N136|N151|N16|N2884|N2885|N2886|N3000|N3001|N3090|N390')

# # For uti, exclude any listed ICD-10 with the diagnosis codes below by overriding any value of 1 to 0 in the 'uti' column.
# for column in diagnoses:
#     final_cohort_file.loc[final_cohort_file[column].str.contains('N110|N111|N118|N1370|N1371|N13721|N13722|N13729|N13731|N13732|N13739|N139|Q600|Q601|Q602|Q603|Q604|Q605|Q606|Q61|Q62|Q63|Q64'), 'uti'] = 0

# # Special exclusions for PQI 12, any with ICD-10 diagnosis code or ICD-10 procedural code that indicate immunocomprimised state.
# for column in diagnoses:
#     final_cohort_file.loc[final_cohort_file[column].str.contains('B20|E41|B59|E42|C802|E43|C888|I120|C9440|I1311|C9441|I132|C9442|K912|C946|N185|D4622|N186|D4701|T8600|D4702|T8601|D4709|T8602|D471|T8603|D479|T8609|D47Z1|T8610|D47Z2|T8611|D47Z9|T8612|D6109|T8613|D61810|T8619|v2020|D61811|T8620|D61818|T8621|D700|T8622|D701|T8623|D702|T86290|D704|T86298|D708|T8630|D709|T8631|D71|T8632|D720|T8633|D72810|T8639|D72818|T8640|D72819|T8641|D7381|T8642|D7581|T8643|D761|T8649|D762|T865|D763|T86810|D800|T86811|D801|T86812|D802|T86818|D803|T86819|D804|T86830|D805|T86831|D806|T86832|D807|T86838|D808|T86839|v2020|D809|T86850|D810|T86851|D811|T86852|D812|T86858|D8130|T86859|D8131|T86890|D8132|T86891|D8139|T86892|D814|T86898|D816|T86899|D817|T8690|D8189|T8691|D819|T8692|D820|T8693|D821|T8699|D822|Z4821|D823|Z4822|D824|Z4823|D828|Z4824|D829|Z48280|v2020|D830|Z48288|D831|Z48290|D832|Z48298|D838|Z4901|D839|Z4902|D840|Z4931|D841|Z4932|D848|Z940|D849|Z941|D893|Z942|D89810|Z943|D89811|Z944|D89812|Z9481|D89813|Z9482|D8982|Z9483|D8989|Z9484|D899|Z9489|E40|Z992'), 'uti'] = 0

# for column in procedures:
#     final_cohort_file.loc[final_cohort_file[column].str.contains('30233Y4|02YA0Z2|30240AZ|0BYC0Z0|30240G0|0BYC0Z2|30240G1|0BYD0Z0|30240G2|0BYD0Z2|30240G3|0BYF0Z0|30240G4|0BYF0Z2|30240U2|0BYG0Z0|30240U3|0BYG0Z2|30240U4|0BYH0Z0|30240X0|0BYH0Z2|30240X1|0BYJ0Z0|30240X2|0BYJ0Z2|30240X3|0BYK0Z0|30240X4|0BYK0Z2|30240Y0|0BYL0Z0|30240Y1|0BYL0Z2|30240Y2|0BYM0Z0|30240Y3|0BYM0Z2|30240Y4|0DY50Z0|30243AZ|0DY50Z2|30243G0|0DY60Z0|30243G1|0DY60Z2|30243G2|0DY80Z0|30243G3|0DY80Z2|30243G4|0DYE0Z0|30243U2|0DYE0Z2|30243U3|0FY00Z0|30243U4|0FY00Z2|30243X0|0FYG0Z0|30243X1|0FYG0Z2|30243X2|0TY00Z0|30243X3|0TY00Z2|30243X4|0TY10Z0|30243Y0|0TY10Z2|30243Y1|0WY20Z0|30243Y2|0XYJ0Z0|30243Y3|0XYK0Z0|30243Y4|30230AZ|3E03005|30230G0|3E0300M|30230G1|3E0300P|30230G2|3E030U1|30230G3|3E030WL|30230G4|3E03305|30230U2|3E0330M|30230U3|3E0330P|30230U4|3E033U1|30230X0|3E033WL|30230X1|3E04005|30230X2|3E0400M|30230X3|3E0400P|30230X4|3E040WL|30230Y0|3E04305|30230Y1|3E0430M|30230Y2|3E0430P|30230Y3|3E043WL|30230Y4|3E0A305|30233AZ|3E0A30M|30233G0|3E0D705|30233G1|3E0G705|30233G2|3E0J3U1|30233G3|3E0J7U1|30233G4|3E0J8U1|30233U2|XW03351|30233U3|XW033B3|30233U4|XW033Q5|30233X0|XW033S5|30233X1|XW04351|30233X2|XW043B3|30233X3|XW043Q5|30233X4|XW043S5|30233Y0|XW0DXJ5|30233Y1|XW0DXL5|30233Y2|XW0DXR5|30233Y3|XW0DXT5|XW0DXV5'), 'uti'] = 0

# # -- PQI 14 -- #
# # Uncontrolled diabetes
# icd10_to_column(final_cohort_file, diagnoses, 'uncontrolled_diabetes', 'E1065|E1165|E10649|E11649')

# # -- PQI 15 -- #
# # Asthma in younger adults
# icd10_to_column(final_cohort_file, diagnoses, 'asthma_in_younger_adults', 'J4520|J4521|J4522|J4530|J4531|J4532|J4540|J4541|J4542|J4550|J4552|J45901|J45902|J45909|J45990|J45991|J45998')

# # For asthma_in_younger_adults, exclude any listed ICD-10 for cystic fibrosis with the diagnosis codes below by resetting any value of 1 to 0 in the 'asthma_in_younger_adults' column.
# for column in diagnoses:
#     final_cohort_file.loc[final_cohort_file[column].str.contains('E840|E8411|E8419|E848|E849|E8483|E84841|E849|J8483|J84841|J84842|J84843|J84848|P270|P271|P278|P279|Q254|Q311|Q312|Q313|Q315|Q318|Q319|Q320|Q321|Q322|Q323|Q324|Q33|Q34|Q39|Q693'), 'asthma_in_younger_adults'] = 0

# # -- PQI 16 -- #
# # Lower-extremity amputation among diabetics
# # ICD-10 procedural code for amputation WITH ICD-10 code for diabetes (must have a diagnosis from each)
# for dx_column in diagnoses:
#     for pr_column in procedures:
#         final_cohort_file.loc[(final_cohort_file[dx_column].str.contains('E1010|E1011|E1021|E1022|E1029|E10311|E10319|E10321|E10329|E10331|E10339|E10341|E10349|E10351|E10359|E1036|E1039|E1040|E1041|E1042|E1043|E1044|E1049|E1051|E1052|E1059|E10610|E10618|E10620|E10621|E10622|E10628|E10630|E10638|E10641|E10649|E1065|E1069|E108|E109|E1100|E1101|E1121|E1122|E1129|E11311|E11319|E11321|E11329|E11331|E11339|E11341|E11349|E11351|E11359|E1136|E1139|E1140|E1141|E1142|E1143|E1144|E1149|E1151|E1152|E1159|E11610|E11618|E11620|E11621|E11622|E11628|E11630|E11638|E11641|E11649|E1165|E1169|E118|E119|E1300|E1301|E1310|E1311|E1321|E1322|E1329|E13311|E13319|E13321|E13329|E13331|E13339|E13341|E13349|E13351|E13359|E1336|E1339|E1340|E1341|E1342|E1343|E1344|E1349|E1351|E1352|E1359|E13610|E13618|E13620|E13621|E13622|E13628|E13630|E13638|E13641|E13649|E1365|E1369|E138|E139')) & (final_cohort_file[pr_column].str.contains('0Y620ZZ|0Y630ZZ|0Y640ZZ|0Y670ZZ|0Y680ZZ|0Y6C0Z1|0Y6C0Z2|0Y6C0Z3|0Y6D0Z1|0Y6D0Z2|0Y6D0Z3|0Y6F0ZZ|0Y6G0ZZ|0Y6H0Z1|0Y6H0Z2|0Y6H0Z3|0Y6J0Z1|0Y6J0Z2|0Y6J0Z3|0Y6M0Z0|0Y6M0Z4|0Y6M0Z5|0Y6M0Z6|0Y6M0Z7|0Y6M0Z8|0Y6M0Z9|0Y6M0ZB|0Y6M0ZC|0Y6M0ZD|0Y6M0ZF|0Y6N0Z0|0Y6N0Z4|0Y6N0Z5|0Y6N0Z6|0Y6N0Z7|0Y6N0Z8|0Y6N0Z9|0Y6N0ZB|0Y6N0ZC|0Y6N0ZD|0Y6N0ZF|0Y6P0Z0|0Y6P0Z1|0Y6P0Z2|0Y6P0Z3|0Y6Q0Z0|0Y6Q0Z1|0Y6Q0Z2|0Y6Q0Z3|0Y6R0Z0|0Y6R0Z1|0Y6R0Z2|0Y6R0Z3|0Y6S0Z0|0Y6S0Z1|0Y6S0Z2|0Y6S0Z3|0Y6T0Z0|0Y6T0Z1|0Y6T0Z2|0Y6T0Z3|0Y6U0Z0|0Y6U0Z1|0Y6U0Z2|0Y6U0Z3|0Y6V0Z0|0Y6V0Z1|0Y6V0Z2|0Y6V0Z3|0Y6W0Z0|0Y6W0Z1|0Y6W0Z2|0Y6W0Z3|0Y6X0Z0|0Y6X0Z1|0Y6X0Z2|0Y6X0Z3|0Y6Y0Z0|0Y6Y0Z1|0Y6Y0Z2|0Y6Y0Z3')), 'lower_extremity_amputation_among_diabetics'] = 1

# # For lower_extremity_amputation_among_diabetics, exclude any listed ICD-10 procedure for traumatic amputation with the procedure codes below by resetting any value of 1 to 0 in the
# # 'lower_extremity_amputation_among_diabetics' column.
# for column in procedures:
#     final_cohort_file.loc[final_cohort_file[column].str.contains('S98011A|S78012A|S98012A|S78019A|S98019A|S78021A|S98021A|S78022A|S98022A|S78029A|S98029A|S78111A|S98111A|S78112A|S98112A|S78119A|S98119A|S78121A|S98121A|S78122A|S98122A|S78129A|S98129A|S78911A|S98131A|S78912A|S98132A|S78919A|S98139A|S78921A|S98141A|S78922A|S98142A|S78929A|S98149A|S88011A|S98211A|S88012A|S98212A|S88019A|S98219A|S88021A|S98221A|S88022A|S98222A|S88029A|S98229A|S88111A|S98311A|S88112A|S98312A|S88119A|S98319A|S88121A|S98321A|S88122A|S98322A|S88129A|S98329A|S88911A|S98911A|S88912A|S98912A|S88919A|S98919A|S88921A|S98921A|S88922A|S98922A|S88929A|S98929A'), 'lower_extremity_amputation_among_diabetics'] = 0

## Calculated fields

In [31]:
# This function will take care of creating a new column for followup diagnoses/procedures as well as a new column for total diagnoses/procedures, per type of infection, use, etc.
# - df = pass in the dataframe that has the data that you want to create the columns in.
# - columns to sum = a list of column names that will be used to flag a 0 or 1 for if the patient has that type of diagnosis/procedure.
# - followup_column_name = a string reference of what the column with the flagged outcomes will be named.
# - total_column_name = a string reference of what the column with the total per patient will be named.
# Must return the dataframe once it has been updated, otherwise localized variables will be lost.
def followup(df, columns_to_sum, followup_column_name, total_column_name):

    # A list of all column names.
    all_columns = df.columns.to_list()

    # Initialize a list to hold the numerical position of the columns and a counter.
    index_column_positions = []
    y = 0

    # Create a list of columns only if they include one of the columns that is in the columns_to_sum.
    for column in all_columns:
        for name in columns_to_sum:
            if name == column:
                index_column_positions.append(y)
        y += 1

    # A new column that sums up the values in all of the columns_to_sum columns so we can create the temp_total column. If this column is >= 1 then it means that the 
    # diagnosis/procedure/infection/usage exists for that patient in that visit. If it's 0, then it means they did not have that diagnosis/procedure/infection/usage on that visit.
    df['temp_total'] = df.iloc[:, index_column_positions[0]:index_column_positions[-1]+1].sum(axis=1)

    # Start with a placeholder for nrd_visitlink that is made up ('aaaaaa'). This will hold the previous rows nrd_visitlink in each iteration.
    # Keep a variable to track whether or not it is before or after the index hospitalization.
    # row_num keeps track of which row we are on.
    id_placeholder = 'aaaaaa'
    after_index = 'yes'
    row_num = 0

    df[followup_column_name] = 0

    # This loop goes through each row and compares the nrd_visitlink with that of the previous row. 
    for row in df.itertuples(index=False):
        id_current = row.nrd_visitlink
        # If the current and previous nrd_visitlink are not the same and it is the patient's index hospitalization and temp_total >= 1, then add a 1 to the followup_column_name
        # column. Set after_index to 'yes'.
        if id_current != id_placeholder and row.ibd_index_hospitalization == 1 and row.temp_total >= 1:
            df.at[row_num, followup_column_name] = 1
            id_placeholder = id_current
            after_index = 'yes'
        # If the current and previous nrd_visitlink are not the same and it is the patient's index hospitalization and temp_total < 1, then add a 0 to the followup_column_name
        # column. Set after_index to 'yes'.
        elif id_current != id_placeholder and row.ibd_index_hospitalization == 1 and row.temp_total < 1:
            df.at[row_num, followup_column_name] = 0
            id_placeholder = id_current
            after_index = 'yes'
        # If the current and previous nrd_visitlink are not the same and it is not the patient's index hospitalization, then add a 0 to the followup_column_name column. Set
        # after_index to 'no'.
        elif id_current != id_placeholder and row.ibd_index_hospitalization == 0:
            df.at[row_num, followup_column_name] = 0
            id_placeholder = id_current
            after_index = 'no'
        # If the current and previous nrd_visitlink are the same and it is not the patient's index hospitalization, then add a 0 to the followup_column_name column. Set after_index
        # to 'no'.
        elif id_current == id_placeholder and row.ibd_index_hospitalization == 0 and after_index == 'no':
            df.at[row_num, followup_column_name] = 0
            id_placeholder = id_current
            after_index = 'no'
        # If the current and previous nrd_visitlink are the same and it is the patient's index hospitalization and temp_total >= 1, then add a 1 to the followup_column_name column.
        # Set after_index to 'yes'.
        elif id_current == id_placeholder and row.ibd_index_hospitalization == 1 and row.temp_total >= 1:
            df.at[row_num, followup_column_name] = 1
            id_placeholder = id_current
            after_index = 'yes'
        # If the current and previous nrd_visitlink are the same and it is the patient's index hospitalization and temp_total < 1, then add a 0 to the followup_column_name column.
        # Set after_index to 'yes'.
        elif id_current == id_placeholder and row.ibd_index_hospitalization == 1 and row.temp_total < 1:
            df.at[row_num, followup_column_name] = 0
            id_placeholder = id_current
            after_index = 'yes'
        # If the current and previous nrd_visitlink are the same and temp_total >= 1, then add a 1 to the followup_column_name column.
        elif id_current == id_placeholder and after_index == 'yes' and row.temp_total >= 1:
            df.at[row_num, followup_column_name] = 1
            id_placeholder = id_current
         # If the current and previous nrd_visitlink are the same and temp_total < 1, then add a 0 to the followup_column_name column.
        elif id_current == id_placeholder and after_index == 'yes' and row.temp_total < 1:
            df.at[row_num, followup_column_name] = 0
            id_placeholder = id_current
        row_num += 1

    # Drop the 'temp_total' column since we no longer need it.
    df = df.drop(columns=['temp_total'])

    # Use pivot_table with aggregate function 'sum' to essentially group by the nrd_visitlinks and sum the total number of diagnosis/procedure/infection/usage for each patient.
    pivot_df = df.pivot_table(index='nrd_visitlink', values=followup_column_name, aggfunc='sum').reset_index()

    # Since the pivot table only has unique patient nrd_visitlink's and totals per patient, create a dictionary to store each patient and their total number of 
    # diagnosis/procedure/infection/usage.
    patient_totals = {}
    for row in pivot_df.itertuples(index=False):
        patient_totals.update({row.nrd_visitlink: row[1]})

    # Create a new column called total_column_name by mapping the nrd_visitlinks values from the dicionary patient_totals.
    df[total_column_name] = df['nrd_visitlink'].map(patient_totals)

    # Print statment so we know it's done. Before printing, split the string into individual words, join into single string with spaces, split on 'during', trim whitespaces.
    followup_column_name = followup_column_name.split("_")
    followup_column_name = " ".join(followup_column_name).split("during")
    print(f'Completed columns for {followup_column_name[0].strip()}.')

    # Return the dataframe with the new columns, otherwise we will not have any of the columns when we move forward.
    return df

In [32]:
# First create a list of the required column names.
# Then call the function created above to populate the calculated field columns.

# -- DIAGNOSIS CODES -- #

# -- SERIOUS INFECTIONS -- #
serious_infections = ['intestinal_abscess', 'meningitis', 'encephalitis', 'endocarditis', 'pneumonia', 'pyelonephritis', 'septic_arthritis_osteomyelitis', 'septicemia_or_bacteremia', 'clostridium_difficile']

final_cohort_file = followup(final_cohort_file, serious_infections, 'serious_infections_during_followup', 'total_serious_infection_admissions')


# -- OPPORTUNISTIC INFECTIONS -- #
opportunistic_infections = ['pulmonary_tuberculosis', 'atypical_mycobacteria', 'cryptococcosis_aspergillosis_histoplasmosis', 'listeriosis', 'leishmaniasis', 'pneumocystis_jiroveci_pneumonia']

final_cohort_file = followup(final_cohort_file, opportunistic_infections, 'opportunistic_infections_during_followup', 'total_opportunistic_infection_admissions')

# -- CARDIAC -- #
cardiac = ['acute_myocardial_infarction', 'heart_failure', 'unstable_angina', 'arrhythmia', 'cardiac_arrest']

final_cohort_file = followup(final_cohort_file, cardiac, 'cardiac_during_followup', 'total_cardiac_admissions')


# -- VENOUS THROMBOEMBOLISM -- #
venous_thromboembolism = ['vte_diagnosis_pe_or_dvt', 'history_of_vte']

final_cohort_file = followup(final_cohort_file, venous_thromboembolism, 'venous_thromboembolism_during_followup', 'total_venous_thromboembolism_admissions')


# -- STEROID USE -- #
steroid_use = ['systemic_steroid_use']

final_cohort_file = followup(final_cohort_file, steroid_use, 'steroid_use_during_followup', 'total_steroid_use_admissions')


# -- CHRONIC PAIN -- #
chronic_pain = ['chronic_pain']

final_cohort_file = followup(final_cohort_file, chronic_pain, 'chronic_pain_during_followup', 'total_chronic_pain_admissions')


# -- OPIOID OVERUSE -- #
opioid_overuse = ['opioid_overuse']

final_cohort_file = followup(final_cohort_file, opioid_overuse, 'opioid_overuse_during_followup', 'total_opioid_overuse_admissions')


# -- PSYCHIATRIC BURDEN -- #
psychiatric_burden = ['depression', 'anxiety_disorder', 'bipolar_disorder'] # not including schizophrenia

final_cohort_file = followup(final_cohort_file, psychiatric_burden, 'psychiatric_burden_during_followup', 'total_psychiatric_burden_admissions')

# -- SOCIAL DETERMINANTS OF HEALTH -- #
social_determinants_of_health = ['problems_related_to_education_and_literacy', 'problems_related_to_employment_and_unemployment', 'occupational_exposure_to_risk_factors', 'problems_related_to_housing_and_economic_circumstances', 'problems_related_to_social_environment', 'problems_related_to_upbringing', 'other_problems_related_to_primary_support_group_including_family_circumstances', 'problems_related_to_certain_psychosocial_circumstances', 'problems_related_to_other_psychosocial_circumstances']

final_cohort_file = followup(final_cohort_file, serious_infections, 'social_determinants_of_health_during_followup', 'total_social_determinants_of_health_admissions')

# -- PROCEDURE CODES -- #

# -- SURGERY -- #

# -- INTESTINAL SURGERY -- #
intestinal_surgery = ['intestinal_surgery']

final_cohort_file = followup(final_cohort_file, intestinal_surgery, 'intestinal_surgery_during_followup', 'total_intestinal_surgery_admissions')

# -- PROCEDURES -- #

# -- FLEXIBLE SIGMOIDOSCOPY/COLONOSCOPY -- #
flexible_sigmoidoscopy_colonoscopy = ['flexible_sigmoidoscopy_colonoscopy']

final_cohort_file = followup(final_cohort_file, flexible_sigmoidoscopy_colonoscopy, 'flexible_sigmoidoscopy_colonoscopy_during_followup', 'total_flexible_sigmoidoscopy_colonoscopy_admissions')

# -- BLOOD TRANSFUSION -- #
blood_transfusion = ['blood_transfusion']

final_cohort_file = followup(final_cohort_file, blood_transfusion, 'blood_transfusion_during_followup', 'total_blood_transfusion_admissions')

# -- PARENTERAL OR ENTERAL NUTRITION -- #
parenteral_or_enteral_nutrition = ['parenteral_or_enteral_nutrition']

final_cohort_file = followup(final_cohort_file, parenteral_or_enteral_nutrition, 'parenteral_or_enteral_nutrition_during_followup', 'total_parenteral_or_enteral_nutrition_admissions')

Completed columns for serious infections.
Completed columns for opportunistic infections.
Completed columns for cardiac.
Completed columns for venous thromboembolism.
Completed columns for steroid use.
Completed columns for chronic pain.
Completed columns for opioid overuse.
Completed columns for psychiatric burden.
Completed columns for social determinants of health.
Completed columns for intestinal surgery.
Completed columns for flexible sigmoidoscopy colonoscopy.
Completed columns for blood transfusion.
Completed columns for parenteral or enteral nutrition.


In [115]:
# Save the file.
final_cohort_file.to_csv("NRD_2017_Defined_Cohort.csv", index=False)

In [33]:
# This code can be used to check values and counts in any column.

# # Print out the number of times each value shows up. Value is the different values that are in the specified column and Counts is the number of times a specific value appears in
# # that column.
# values, counts = np.unique(final_cohort_file['depression'], return_counts=True)
# print('Values:  Counts:')
# for number in values:
#     print(f'{values[number]}        {counts[number]}')

In [2]:
final_cohort_file = pd.read_csv("NRD_2017_Defined_Cohort.csv")
final_cohort_file

,key_nrd,age,aweekend,died,discwt,dispuniform,dmonth,dqtr,drg,drgver,drg_nopoa,i10_dx1,i10_dx2,i10_dx3,i10_dx4,i10_dx5,i10_dx6,i10_dx7,i10_dx8,i10_dx9,i10_dx10,i10_dx11,i10_dx12,i10_dx13,i10_dx14,i10_dx15,i10_dx16,i10_dx17,i10_dx18,i10_dx19,i10_dx20,i10_dx21,i10_dx22,i10_dx23,i10_dx24,i10_dx25,i10_dx26,i10_dx27,i10_dx28,i10_dx29,i10_dx30,i10_dx31,i10_dx32,i10_dx33,i10_dx34,i10_dx35,i10_dx36,i10_dx37,i10_dx38,i10_dx39,i10_dx40,elective,female,hcup_ed,hosp_nrd_x,los,mdc,mdc_nopoa,i10_ndx,i10_npr,nrd_daystoevent,nrd_stratum_x,nrd_visitlink,pay1,pl_nchs,i10_pr1,i10_pr2,i10_pr3,i10_pr4,i10_pr5,i10_pr6,i10_pr7,i10_pr8,i10_pr9,i10_pr10,i10_pr11,i10_pr12,i10_pr13,i10_pr14,i10_pr15,i10_pr16,i10_pr17,i10_pr18,i10_pr19,i10_pr20,i10_pr21,i10_pr22,i10_pr23,i10_pr24,i10_pr25,prday1,prday2,prday3,prday4,prday5,prday6,prday7,prday8,prday9,prday10,prday11,prday12,prday13,prday14,prday15,prday16,prday17,prday18,prday19,prday20,prday21,prday22,prday23,prday24,prday25,rehabtransfer,resident,samedayevent,totchg,year_x,zipinc_qrtl,dxver,prver,hosp_bedsize,h_contrl,hosp_urcat4,hosp_ur_teach,nrd_stratum_y,n_disc_u,n_hosp_u,s_disc_u,s_hosp_u,total_disc,year_y,aprdrg,aprdrg_risk_mortality,aprdrg_severity,hosp_nrd_y,next_nrdday,days_to_readmission,readmission_diagnosis,readmission_los,preventable_admission,pqi_1,pqi_3,pqi_5,pqi_7,pqi_8,pqi_11,pqi_12,pqi_14,pqi_15,pqi_16,pqi_5_15_exclusion,pqi_7_medical_exclusions,pqi_7_procedural_exclusion,pqi_7_8_procedural_exclusion,pqi_11_exclusion,pqi_12_exclusion,pqi_immunocompromise_medical_exclusion,pqi_immunocompromise_procedural_exclusion,pqi_16_exclusion_procedure,pqi_16_amputation_procedure,medical_admission,hfrs_score,hfrs_severity,cc_score,cc_severity,cc_MI,cc_CHF,cc_PVD,cc_CVD,cc_dementia,cc_COPD,cc_rheum,cc_peptic_ulcer,cc_mild_liver_disease,cc_diabetes_wo_complications,cc_diabetes_w_complications,cc_plegia,cc_renal,cc_malignancy,cc_severe_liver,cc_metastasis,cc_AIDS,ibd_diagnosis,ibd_index_hospitalization,intestinal_abscess,meningitis,encephalitis,endocarditis,pneumonia,pyelonephritis,septic_arthritis_osteomyelitis,septicemia_or_bacteremia,clostridium_difficile,pulmonary_tuberculosis,atypical_mycobacteria,cryptococcosis_aspergillosis_histoplasmosis,listeriosis,leishmaniasis,pneumocystis_jiroveci_pneumonia,acute_myocardial_infarction,heart_failure,unstable_angina,arrhythmia,cardiac_arrest,vte_diagnosis_pe_or_dvt,history_of_vte,systemic_steroid_use,chronic_pain,opioid_overuse,depression,anxiety_disorder,bipolar_disorder,schizophrenia,problems_related_to_education_and_literacy,problems_related_to_employment_and_unemployment,occupational_exposure_to_risk_factors,problems_related_to_housing_and_economic_circumstances,problems_related_to_social_environment,problems_related_to_upbringing,other_problems_related_to_primary_support_group_including_family_circumstances,problems_related_to_certain_psychosocial_circumstances,problems_related_to_other_psychosocial_circumstances,intestinal_surgery,flexible_sigmoidoscopy_colonoscopy,blood_transfusion,parenteral_or_enteral_nutrition,serious_infections_during_followup,total_serious_infection_admissions,opportunistic_infections_during_followup,total_opportunistic_infection_admissions,cardiac_during_followup,total_cardiac_admissions,venous_thromboembolism_during_followup,total_venous_thromboembolism_admissions,steroid_use_during_followup,total_steroid_use_admissions,chronic_pain_during_followup,total_chronic_pain_admissions,opioid_overuse_during_followup,total_opioid_overuse_admissions,psychiatric_burden_during_followup,total_psychiatric_burden_admissions,social_determinants_of_health_during_followup,total_social_determinants_of_health_admissions,intestinal_surgery_during_followup,total_intestinal_surgery_admissions,flexible_sigmoidoscopy_colonoscopy_during_followup,total_flexible_sigmoidoscopy_colonoscopy_admissions,blood_transfusion_during_followup,total_blood_transfusion_admissions,parenteral_or_enteral_nutrition_during_followup,total_parenteral_or_enteral_nutrition_admi

# Additional columns to calculate HNHC

## severe_ibd_hospitalization
### - Length of stay >= 7 or surgery during hospitalization
<br></br>
## tot_admits_for_pt
### - Total number of hospitalizations for a given patient (including the index hospitalization)
<br></br>
## tot_hosp_charges_for_pt
### - Total hospital charges across all hospitalizations (sum hospital charges across hospitalizations, including index hospitalization)
<br></br>
## tot_los_pt
### - Total length of stay across all hospitalizations (including the index hospitalization)


In [111]:
# Start with a placeholder for nrd_visitlink that is initialized with the first nrd_visitlink. This will hold the previous rows nrd_visitlink in each iteration.
# Keep a variable to track whether or not it is before or after the index hospitalization. Initialize with 'no'.
# row_num keeps track of which row we are on.
id_placeholder = final_cohort_file['nrd_visitlink'][0]
after_index = 'no'
row_num = 0

# Counter will count the number of times a patient has been admitted starting from the index hospitalization. 
# total_admits will create a mapping dictionary from which we can map the totals back to the dataframe using the nrd_visitlink.
counter = 1
total_admits = {}

# Initialize tot_chrg with 0. This varable will hold the sum of charges for each patient from the index hospitalization to their last.
# total_charges will create a mapping dictionary from which we can map the total charges back to the dataframe using the nrd_visitlink.
tot_chrg = 0
total_charges = {}

# Initialize tot_los with 0. This varable will hold the sum of length of stay for each patient from the index hospitalization to their last.
# total_los will create a mapping dictionary from which we can map the total length of stay back to the dataframe using the nrd_visitlink.
tot_los = 0
total_los = {}

# Store the nrd_visitlink for the last patient in the dataframe for use later.
last_patient = final_cohort_file['nrd_visitlink'][len(final_cohort_file)-1]

# Initialize the severe_ibd_hospitalization column with a 0.
final_cohort_file['severe_ibd_hospitalization'] = 0

# This loop goes through each row and compares the nrd_visitlink with that of the previous row. 
for row in final_cohort_file.itertuples(index=False):
    id_current = row.nrd_visitlink
    # If the last row is not a hospitalization related to the previous patient id, then it must be a new patient with an index hospitalization, fill everything in accordingly.
    # This MUST be the first check in the loop, otherwise it will default to the first elif statement.
    if id_current == last_patient and row.ibd_index_hospitalization == 1:
        if row.los > 6 or row.intestinal_surgery == 1:
            final_cohort_file.at[row_num, 'severe_ibd_hospitalization'] = 1
        total_admits.update({final_cohort_file['nrd_visitlink'][len(final_cohort_file)-2]:counter})
        total_charges.update({final_cohort_file['nrd_visitlink'][len(final_cohort_file)-2]:tot_chrg})
        total_los.update({final_cohort_file['nrd_visitlink'][len(final_cohort_file)-2]:tot_los})
        tot_chrg = 0
        tot_chrg += row.totchg
        tot_los = 0
        tot_los += row.los
        total_charges.update({last_patient:tot_chrg})
        total_admits.update({last_patient:1})
        total_los.update({last_patient:tot_los})
    # If the current and previous nrd_visitlink are not the same and it is the patient's index hospitalization, then check if the length of stay is over 6 days or if they had
    # surgery during this visit. If either is true, then add a 1 to severe_ibd_hospitalization and set after_index to 'yes'.
    elif id_current != id_placeholder and row.ibd_index_hospitalization == 1:
        if row.los > 6 or row.intestinal_surgery == 1:
            final_cohort_file.at[row_num, 'severe_ibd_hospitalization'] = 1
        # Update the total_admits dictionary with the previous patient id and their counter.
        total_admits.update({id_placeholder:counter})
        # Update the total_charges dictionary with the previous patient id and their total charges.
        total_charges.update({id_placeholder:tot_chrg})
        # Update the total_los dictionary with the previous patient id and their total length of stay.
        total_los.update({id_placeholder:tot_los})
        # Update the nrd_visitlink to the current one.
        id_placeholder = id_current
        # Reset total charges and then start adding because this is the index hospitalization.
        tot_chrg = 0
        tot_chrg += row.totchg
        # Reset total length of stay and then start adding because this is the index hospitalization.
        tot_los = 0
        tot_los += row.los
        # Set counter to 1.
        counter = 1
        after_index = 'yes'
    # If the current and previous nrd_visitlink are not the same and it is not the patient's index hospitalization, then set after_index to 'no'.
    elif id_current != id_placeholder and row.ibd_index_hospitalization == 0:
        id_placeholder = id_current
        total_admits.update({id_placeholder:counter})
        total_charges.update({id_placeholder:tot_chrg})
        total_los.update({id_placeholder:tot_los})
        id_previous = id_placeholder
        tot_chrg = 0
        tot_los = 0
        counter = 1
        after_index = 'no'
    # If the current and previous nrd_visitlink are the same and it is not the patient's index hospitalization, then set after_index to 'no'.
    elif id_current == id_placeholder and row.ibd_index_hospitalization == 0 and after_index == 'no':
        id_placeholder = id_current
        tot_chrg = 0
        tot_los = 0
        counter = 1
        after_index = 'no'
    # If the current and previous nrd_visitlink are the same and it is not the patient's index hospitalization but it is after the index hospitalization, then check if the length of
    # stay is over 6 days or if they had surgery during this visit. If either is true, then add a 1 to severe_ibd_hospitalization and set after_index to 'yes'.
    elif id_current == id_placeholder and row.ibd_index_hospitalization == 0 and after_index == 'yes':
        if row.los > 6 or row.intestinal_surgery == 1:
            final_cohort_file.at[row_num, 'severe_ibd_hospitalization'] = 1
        id_placeholder = id_current
        # We are after the index now so we need to start adding up the total charges.
        tot_chrg += row.totchg
        # We are after the index now so we need to start adding up the total length of stay.
        tot_los += row.los
        # We are after the index now so we need to start incrementing the counter.
        counter += 1
        after_index = 'yes'
    # If the current and previous nrd_visitlink are the same and it is the patient's index hospitalization, then check if the length of stay is over 6 days or if they had surgery
    # during this visit. If either is true, then add a 1 to severe_ibd_hospitalization and set after_index to 'yes'.
    elif id_current == id_placeholder and row.ibd_index_hospitalization == 1:
        if row.los > 6 or row.intestinal_surgery == 1:
            final_cohort_file.at[row_num, 'severe_ibd_hospitalization'] = 1
        id_placeholder = id_current
        # Reached the index so now we start summing the total charges.
        tot_chrg += row.totchg
        # Reached the index so now we start summing the total length of stay.
        tot_chrg += row.totchg
        tot_los += row.los
        counter = 1
        after_index = 'yes'
    row_num += 1

# Map the patients totals for each new column based on their nrd_visitlink.
final_cohort_file['tot_admits_for_pt'] = final_cohort_file["nrd_visitlink"].map(total_admits)
final_cohort_file['tot_hosp_charges_for_pt'] = final_cohort_file["nrd_visitlink"].map(total_charges)
final_cohort_file['tot_los_pt'] = final_cohort_file["nrd_visitlink"].map(total_los)

## - Chron's Disease and Ulcerative Colitis

In [35]:
# Create a list to hold all of the positions, counter and row_num will hold column and row values respectively.
diagnoses = []
counter = 0
row_num = 0

# This dictionary holds the fill in values for any blanks or NaN's in the data for the columns beginning with 'i10_dx'.
# If there is no discharge diagnosis, it will be filled in with a value of '' so we can iterate through the rows without any errors below for NaN's.
fill_values = {}

# Looping through to get the column position for every column beginning with 'i10_dx'
for column in final_cohort_file.columns:
    if 'i10_dx' in column:
        diagnoses.append(counter)
        fill_values.update({column: ''})
    counter += 1

# Fill the i10_dx column NaN values with ''.
final_cohort_file = final_cohort_file.fillna(value = fill_values)

# Initialize the two new columns we want to populate with values.
final_cohort_file['CD_patients_only'] = 0
final_cohort_file['UC_patients_only'] = 0

# Go through the dataframe and add a 1 to either column if it is the patients index hospitalization and they have either chron's disease or ulcerative colitis.
for row in tqdm(final_cohort_file.itertuples(index=False)):
    for column in diagnoses:
        if row.ibd_index_hospitalization == 1 and 'K50' in row[column]:
            final_cohort_file.at[row_num, 'CD_patients_only'] = 1
        if row.ibd_index_hospitalization == 1 and 'K51' in row[column]:
            final_cohort_file.at[row_num, 'UC_patients_only'] = 1
    row_num += 1

139286it [00:07, 18355.58it/s]
